
# <span style="color:rgb(213,80,0)">README RAVEN GEMs</span>

Author: Vi Varga


Work Period: August 2025

## Description

This MATLAB Live script can create the GEMs that were used for the MCMMs used in the `MCMMs__README_Tracking.ipynb` Jupyter Notebook.


All GEMs in this project were created using the \[RAVEN Toolbox\](https://github.com/SysBioChalmers/RAVEN).

## Setup

The MATLAB environment was set up for the RAVEN Toolbox's use according to the instructions on the Wiki, here: [https://github.com/SysBioChalmers/RAVEN/wiki](https://github.com/SysBioChalmers/RAVEN/wiki)


Notes on RAVEN Toolbox files:

-  All files are stored locally at: `C:\\Users\\viragv\\AppData\\Roaming\\MathWorks\\'MATLAB Add-Ons'\\Collections\\'RAVEN Toolbox'\\`
-  `Within the above directory, files related to the [tutorials](https://github.com/SysBioChalmers/RAVEN/wiki/Tutorials) are found in the `tutorial` directory`

In [1]:
% testing GEM creation with RAVEN
%cd('C:\Users\viragv\AppData\Roaming\MathWorks\'MATLAB Add-Ons'\Collections\'RAVEN Toolbox'\tutorial\')
%tutorial5 % example model reconstruction from KEGG data
%tutorial6 % example model reconstruction from KEGG & MetaCyc data


## Data wrangling

The primary project working directory for this project is: C:/Users/viragv/Documents/ChalmersG/Courses/FKBT190\_Adv\_Biosci\_Tech/MCMMs\_Vaginomics/

-  The Data/ subdirectory contains all protein FASTAs and GEMS
-  The Results/ subdirectory contains the MCMMs

Based on `tutorial6`, the RAVEN Toolbox requires protein FASTA files for the organisms of interest in order to generate the GEMs. These were downloaded from the NCBI as follows:

-  *Trichomonas vaginalis*: [https://www.ncbi.nlm.nih.gov/datasets/genome/GCF\_026262505.1/](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_026262505.1/)
-  *Prevotella bivia*: [https://www.ncbi.nlm.nih.gov/datasets/genome/GCF\_000262545.1/](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000262545.1/)
-  *Gardnerella vaginalis*: [https://www.ncbi.nlm.nih.gov/datasets/genome/GCF\_001042655.1/](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_001042655.1/)
-  *Lactobacillus iners*: [https://www.ncbi.nlm.nih.gov/datasets/taxonomy/147802/](https://www.ncbi.nlm.nih.gov/datasets/taxonomy/147802/)
-  *Lactobacillus crispatus*: [https://www.ncbi.nlm.nih.gov/datasets/taxonomy/47770/](https://www.ncbi.nlm.nih.gov/datasets/taxonomy/47770/)
-  Note: A human GEM was not generated, as the SysBio Human\-GEM is available from GitHub: [https://github.com/SysBioChalmers/Human\-GEM](https://github.com/SysBioChalmers/Human-GEM)

For all protein FASTA datasets, I used the GCF dataset version (GCF is RefSeq; GCA is GenBank).


The FASTA headers were cleaned up for ease of use in GEM generation with the `assignFASTAheaders_GEMs.py` script, like so:


`# gardnerella vaginalis`


`python assignFASTAheaders_GEMs.py Data/gardnerella_vaginalis_GCF_001042655-1_prots.faa Data/GEM_EncodingSummary.txt`


`# lactobacillus crispatus`


`python assignFASTAheaders_GEMs.py Data/lactobacillus_crispatus_GCF_009769205-1_prots.faa Data/GEM_EncodingSummary.txt`


`# lactobacillus iners`


`python assignFASTAheaders_GEMs.py Data/lactobacillus_iners_GCF_011058775-1_prots.faa Data/GEM_EncodingSummary.txt`


`# prevotella bivia`


`python assignFASTAheaders_GEMs.py Data/prevotella_bivia_GCF_000262545-1_prots.faa Data/GEM_EncodingSummary.txt`


`# trichomonas vaginalis`


`python assignFASTAheaders_GEMs.py Data/trichomonas_vaginalis_GCF_026262505-1_prots.faa Data/GEM_EncodingSummary.txt`



In [2]:
%% Environment setup

% set the working directory with
%cd('C:\Users\viragv\Documents\ChalmersG\Courses\FKBT190_Adv_Biosci_Tech\MCMMs_Vaginomics\Data\');
% check that this worked correctly with
%disp(pwd);
% add the script directory to teh PATH
addpath('C:\Users\viragv\Documents\ChalmersG\Courses\FKBT190_Adv_Biosci_Tech\MCMMs_Vaginomics\Scripts\');
% load KEGG database to use for gap-filling
modelKEGG = getModelFromKEGG()

Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
modelKEGG = struct with fields:
             id: 'KEGG'
           name: 'Automatically generated from KEGG database'
           rxns: {11588x1 cell}
       rxnNames: {11588x1 cell}
        eccodes: {11588x1 cell}
     subSystems: {11588x1 cell}
     rxnMiriams: {11588x1 cell}
       rxnNotes: {11588x1 cell}
              S: [9560x11588 double]
           mets: {9560x1 cell}
            rev: [11588x1 double]
             ub: [11588x1 double]
             lb: [11588x1 double]
              c: [11588x1 double]
              b: [9560x1 double]
          genes: {6543571x1 cell}
     rxnGeneMat: [11588x6543571 double]
       metNames: {9560x1 cell}
    metFormulas: {9560x1 cell}
         inchis: {9560x1 cell}
     metMiriams: {9560x1 cell}
          comps: {'s'}
      compNames: {'System'}
       metComps: [9560x1 double]
     annota

In [3]:
%clear; % clear environment variables
%clc % clear the command window


## Small helper functions

Minor cleaning functions used throughout the script:


In [4]:
% grRules continues to cause problems
function model = ensureGrRules(model)
    % Ensure model has grRules and matches rxns length
    if ~isfield(model,'grRules') || numel(model.grRules) ~= numel(model.rxns)
        model.grRules = repmat({''}, numel(model.rxns), 1);
        fprintf('Patched grRules to match %d rxns\n', numel(model.rxns));
    end
end

% ensuring reversibility
function model = ensureRevField(model)
    if ~isfield(model, 'rev')
        % infer reversibility from bounds
        model.rev = model.lb < 0;
        fprintf('Created rev field with %d reversible reactions\n', sum(model.rev));
    else
        fprintf('Model already has rev field\n');
    end
end

% check for dead-end metabolites
function deadMets = quickDeadEnds(model)
    deadMets = {};
    for i = 1:numel(model.mets)
        involved = find(model.S(i,:)); % reactions involving this metabolite
        if isempty(involved)
            deadMets{end+1} = model.mets{i};
        end
    end
    deadMets = deadMets(:);
end


## GEM generation with RAVEN

Below, I'll generate and quality\-check draft GEMs for the organisms of interest.

## Draft Lactobacillus crispatus GEM

In [5]:
%   Reconstruct a combined draft GEM from KEGG and MetaCyc pathway databases.
%   Based on RAVEN Toolbox tutorial6

%Before reconstruction, a FASTA file with protein sequences of the target
%organism needs to be prepared.
%This was doen by downloading sequences from the NCBI & processing them
%with the assignFASTAheaders_GEMs.py script, as described above.

%Generate the two draft models from KEGG database. The first one is
%reconstructed from the genome information annotated by KEGG. Since KEGG
%provides genome annotations for over 5000 organisms, their GEMs can thus
%be generated without providing protein sequences while only using the
%organismID. The organisms with KEGG annotation and their associated ids
%(e.g. 'lcr' for Lactobacillus crispatus) can be found from here:
%https://www.kegg.jp/kegg/catalog/org_list.html.
%Generate a draft model using KEGG annotation and exclude incomplete
%reactions and reactions with undefined stoichiometry
LcrispatusKEGGAnnotation=getKEGGModelForOrganism('lcr','','','',1,0,0);

*** The model reconstruction from KEGG based on the annotation available for KEGG Species lcr ***
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Pruning the model from non-lcr genes... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Constructing GPR associations and annotations for the model... COMPLETE
*** Model reconstruction complete ***

In [6]:
%Generate the second KEGG-based draft model based on sequence homology to
%KEGG Ortholog sequence clusters while excluding incomplete reactions and
%the ones with undefined stoichiometry. Type "help getKEGGModelForOrganism"
%to see the detailed instructions for the choice of different parameters.
%The default values for homology search are used because they have been
%optimized for the best performance.
LcrispatusKEGGHomology=getKEGGModelForOrganism('LcrispatusKEGGHMMs','lactobacillus_crispatus_GCF_009769205-1_prots_edit.fasta','prok90_kegg105','',1,0,0);

*** The model reconstruction from KEGG based on the protein homology search against KEGG Orthology specific HMMs ***
NOTE: Found prok90_kegg105 directory with pre-trained HMMs, it will therefore be used during reconstruction
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Generating the KEGG Orthology specific multi-FASTA files... COMPLETE
Performing clustering and multiple alignment for KEGG Orthology specific protein sets... COMPLETE
Training the KEGG Orthology specific HMMs... COMPLETE
Querying the user-specified FASTA file against the KEGG Orthology specific HMMs... COMPLETE
Parsing the HMM

In [7]:
%De novo reconstruction from MetaCyc should take about 10 minutes, while
%both reconstructions from KEGG may take up to 50-60 minutes in overall
%depending on the computer hardware and the system used. Given that KEGG
%and MetaCyc databases are formulated in different ways; KEGG relies on
%sequence-based annotation, while MetaCyc collects only experimentally
%verified pathways. Therefore, integration of MetaCyc- and KEGG-based draft
%models could have a better coverage of the metbolism for the target
%organism.

%At first, the two KEGG-based models can be directly merged
LcrispatusKEGGDraftModel=mergeModels({LcrispatusKEGGAnnotation LcrispatusKEGGHomology});

disp(numel(LcrispatusKEGGHomology.rxns)+numel(LcrispatusKEGGAnnotation.rxns));

        1353

In [8]:
disp(numel(LcrispatusKEGGDraftModel.rxns));

        1353

In [9]:
%By checking the reaction number, it can be seen that reaction number in
%the merged model equals adding up the reaction numbers in homology and
%annotation KEGG draft models. And there are duplicated reactions in this
%merged model.

In [10]:
%Merge the duplicated reactions into one and combine multiple iso-enzymes
%into a single grRules. The expandModel and contractModel functions are
%utilised, see their instructions for details.
LcrispatusKEGGDraftModel=expandModel(LcrispatusKEGGDraftModel);
LcrispatusKEGGDraftModel=contractModel(LcrispatusKEGGDraftModel);


At this point, a draft GEM has been created for *L. crispatus*, merging the information from MetaCyc and KEGG. While it needs to be improved, I'll save it for now in Excel format to avoid having to recreate it each time.


In [11]:
exportToExcelFormat(LcrispatusKEGGDraftModel, 'Lcrispatus_draftGEM_v1.xlsx');
save('Lcrispatus_draftGEM_v1.mat', 'LcrispatusKEGGDraftModel');

## Sanity checks

The merged GEM likely has compatibility issues between the MetaCyc and KEGG reactions, so I need to clean those up:


In [12]:
%LcrispatusCombinedDraftModel = load('Lcrispatus_draftGEM_v1.mat');
LcrispatusCleanDraftModel = cleanGrRulesGenes(LcrispatusKEGGDraftModel);
LcrispatusCleanDraftModel = mergeDuplicateMetNames(LcrispatusCleanDraftModel);
LcrispatusCleanDraftModel = fixSubsystemsField(LcrispatusCleanDraftModel);


The preliminary model should now be ready for export.


In [13]:
exportModel(LcrispatusCleanDraftModel, 'Lcrispatus_draftGEM_v2.xml');

    C01977
    C04157
    C04432
    C17324
    C19722
    C19723
    C20451
    C20650
    C21582
    C20751
    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [14]:
exportToExcelFormat(LcrispatusCleanDraftModel, 'Lcrispatus_draftGEM_v2.xlsx');
save('Lcrispatus_draftGEM_v2.mat', 'LcrispatusCleanDraftModel');


Saved in SBML standard GEM format.

## Quality checks & improvements

First, import the saved model:


In [15]:
% Load the model from SBML (.xml)
%LcrispatusCleanDraftModel = importModel('Lcrispatus_draftGEM_v2.xml');


Now start quality checks:


In [16]:
%Check out if the newly generated LcrispatusCombinedDraftModel could grow
sol=solveLP(LcrispatusCleanDraftModel);
disp(sol.f);

     0

In [17]:
%The results indicate that this new model is non-functional
% ie., biomass = 0, which means there is no growth


A solution of 0 likely means that this model as it exists is non\-functional. This is most likely due to one of the following:

-  The biomass reaction is incomplete, contains blocked metabolites, or requires compounds that the model cannot synthesize
-  The constraints prevent flux to biomass even though the model could grow.

Possible solutions/things to check include:

-  Perform **gap filling** to see if missing reactions block biomass precursors.
-  Run **metabolite producibility analysis** to identify which biomass components are unreachable.
-  Verify transport/exchange reactions for essential nutrients.
-  Are uptake rates for carbon, nitrogen, phosphorus, sulfur set correctly?
-  Is oxygen (or other electron acceptors) available if the organism is aerobic?
-  Are you using the right biomass reaction for the organism and strain?
## Looking for a biomass reaction

Need to check whether a biomas reaction was generated:


In [18]:
% Look for metabolites whose names include the substring "biomass"
% (case-insensitive because we apply 'lower' before searching).
bioMets = find(contains(lower(LcrispatusCleanDraftModel.metNames), 'biomass'));

% If no biomass-like metabolites are found, raise a warning.
if isempty(bioMets)
    warning('No metabolites with "biomass" in the name were found.');
else
    % Otherwise, search for reactions that involve these biomass metabolites.
    % The stoichiometric matrix S has rows = metabolites, columns = reactions.
    % We check which reactions (columns) have nonzero entries in rows that
    % correspond to the biomass metabolites.
    [rxnIdx, ~] = find(LcrispatusCleanDraftModel.S(bioMets,:) ~= 0);

    % Remove duplicate reaction indices (since a metabolite could appear in
    % multiple positions, or multiple biomass metabolites may map to the same reaction).
    rxnIdx = unique(rxnIdx);

    % Display the identified potential biomass reactions
    disp('Potential biomass reactions:');
    % Display reaction indices, reaction IDs, and human-readable reaction names side by side
    disp([rxnIdx, LcrispatusCleanDraftModel.rxns(rxnIdx)', LcrispatusCleanDraftModel.rxnNames(rxnIdx)']);
end


## Adding a biomass reaction

Initial explorations showed that the current preliminary GEM draft doesn't have a biomass reaction. Hence, biomass will always be 0 until this is added. The following code was generated by ChatGPT; it seeds a biomass reaction from common building blocks like amino acids and nucleotides.


In [19]:
[LcrispatusBiomassDraftModel, biomassRxnID] = addPseudoBiomass_debug_all( ...
    LcrispatusCleanDraftModel, ...                  % Input model
    'BIOMASS_ps', ...                               % ID for new biomass reaction
    [], ...                                         % Use default precursor list
    true, ...                                       % Enable debug printing to console
    'Lcrispatus_biomass_debug.txt', ...             % Save debug log to file
    'Lcrispatus_biomass_metabolites.csv' ...        % Export metabolites to CSV
);

Precursor "L-alanine" → 3 matches found
    Match: "UDP-N-acetylmuramoyl-L-alanine:D-glutamate ligase(ADP-forming)" (ID: R02783)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "D-Glutamate" (ID: C00217)
        Metabolite: "UDP-N-acetylmuramoyl-L-alanine" (ID: C01212)
    Match: "L-Alanine:tRNA(Ala) ligase (AMP-forming)" (ID: R03038)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "tRNA(Ala)" (ID: C01635)
    Match: "UDP-N-acetylmuramate:L-alanine ligase (ADP-forming)" (ID: R03193)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "UDP-N-acetylmuramate" (ID: C01050)
Precursor "L-arginine" → 3 matches found
    Match: "L-Arginine iminohydrolase" (ID: R00552)
        Metabolite: "H2O" (ID: C00001)
        Metabolite: "L-Arginine" (ID: C00062)
    Match: "L-Arginine:tRNA(Arg) ligase (AMP-forming)" (ID: R03646)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: 



Quick cleaning to ensure RAVEN structures are correctly kept:


In [20]:
LcrispatusBiomassDraftModel = syncModelStruct(LcrispatusBiomassDraftModel, 'Lcrispatus_biomass_sync_log.csv');

syncModelStruct: Log saved to "Lcrispatus_biomass_sync_log.csv"

In [21]:
checkModelCoreConsistency(LcrispatusBiomassDraftModel);

--- BASIC DIMENSION CHECKS ---
Reactions: 754
Metabolites: 1010
Genes: 691
Empty grRules in 145 reactions



A few additional problems were discovered when attempting to import this version of the model with COBRApy. The following are additional sanitizations to hopefully avoid those issues:


In [22]:
LcrispatusBiomassDraftModel = sanitizeGEMforSBML(LcrispatusBiomassDraftModel); % Custom function to remove spaces, dashes, etc.

% Fix rxnConfidenceScores field
if ~isfield(LcrispatusBiomassDraftModel,'rxnConfidenceScores')
    % If missing, initialize as zeros
    LcrispatusBiomassDraftModel.rxnConfidenceScores = ...
        zeros(numel(LcrispatusBiomassDraftModel.rxns),1);
else
    % If it exists but is not double, reset as zeros of correct length
    if ~isnumeric(LcrispatusBiomassDraftModel.rxnConfidenceScores)
        LcrispatusBiomassDraftModel.rxnConfidenceScores = ...
            zeros(numel(LcrispatusBiomassDraftModel.rxns),1);
    elseif size(LcrispatusBiomassDraftModel.rxnConfidenceScores,1) ~= numel(LcrispatusBiomassDraftModel.rxns)
        % Resize if length mismatch
        LcrispatusBiomassDraftModel.rxnConfidenceScores = ...
            zeros(numel(LcrispatusBiomassDraftModel.rxns),1);
    end
end


I will now save the new model with the biomass reaction included:


In [23]:
exportModel(LcrispatusBiomassDraftModel, 'Lcrispatus_draftGEM_v3.xml');

    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [24]:
exportToExcelFormat(LcrispatusBiomassDraftModel, 'Lcrispatus_draftGEM_v3.xlsx');
save('Lcrispatus_draftGEM_v3.mat', 'LcrispatusBiomassDraftModel');


## Solving again

Now that a biomass reaction has been added to the model,


In [25]:
%LcrispatusBiomassDraftModel = importModel('Lcrispatus_draftGEM_v3.xml');
sol2 = solveLP(LcrispatusBiomassDraftModel);
disp(sol2.f)

     0



Still 0, so still no growth.

## Stoichiometry & Gap\-filling

If the exchanges aren't the issue, then the next thing to check is the stoichiometry of the biomass reaction, and then to attempt gap\-filling.


In [26]:
% taking a look at the gap situation
%gapReport(LcrispatusBiomassDraftModel);
gapReport_vSimple(LcrispatusBiomassDraftModel);

Dead-end metabolites: 557
    {'C00018'}
    {'C00040'}
    {'C00053'}
    {'C00071'}
    {'C00077'}
    {'C00088'}
    {'C00094'}
    {'C00104'}
    {'C00114'}
    {'C00132'}
    {'C00134'}
    {'C00137'}
    {'C00145'}
    {'C00149'}
    {'C00158'}
    {'C00170'}
    {'C00180'}
    {'C00189'}
    {'C00201'}
    {'C00224'}
    {'C00233'}
    {'C00237'}
    {'C00242'}
    {'C00243'}
    {'C00249'}
    {'C00253'}
    {'C00262'}
    {'C00283'}
    {'C00327'}
    {'C00350'}
    {'C00353'}
    {'C00363'}
    {'C00369'}
    {'C00383'}
    {'C00385'}
    {'C00392'}
    {'C00395'}
    {'C00402'}
    {'C00447'}
    {'C00450'}
    {'C00454'}
    {'C00458'}
    {'C00459'}
    {'C00489'}
    {'C00491'}
    {'C00530'}
    {'C00577'}
    {'C00620'}
    {'C00627'}
    {'C00647'}
    {'C00653'}
    {'C00670'}
    {'C00691'}
    {'C00721'}
    {'C00785'}
    {'C00819'}
    {'C00846'}
    {'C00876'}
    {'C00881'}
    {'C00886'}
    {'C00935'}
    {'C00957'}
    {'C00962'}
    {'C00973'}
    {'C01019'}



The built\-in `gapReport()` program in the RAVEN Toolbox doesn't work for my model, so I've modified `gapReport()`, `checkProduction()` and `haveFlux()` as suggested by ChatGPT, to avoid the issue and still get some results.


In [27]:
% This version produces a more complex result like RAVEN
% But it takes a VERY long time to run, so is commented out here
%gapReport_nosimplify(LcrispatusBiomassDraftModel);


It is clear that there are quite a lot of gaps and problems with this draft model.

## Sorting out exchange reactions

In order for this to be a functional GEM, exchange reactions need to be added.


In [28]:
% Setting up the draft GEM base
%LcrispatusBiomassDraftModel = importModel('Lcrispatus_draftGEM_v3.xml');
% create a copy to avoid overwriting
LcrispatusExchangeModel = LcrispatusBiomassDraftModel;
LcrispatusExchangeModel.id = 'LcrispatusDraft';


Perform some cleaning:


In [29]:
% dealing with grRules
LcrispatusExchangeModel = ensureGrRules(LcrispatusExchangeModel);
modelKEGGgr = ensureGrRules(modelKEGG);

Patched grRules to match 11588 rxns

In [30]:
% further sanitization
modelKEGGgr = padMissingFields(modelKEGGgr, LcrispatusExchangeModel);

Added geneMiriams to KEGG
Added rxnFrom to KEGG
Added metFrom to KEGG
Added geneFrom to KEGG
Added rules to KEGG
Added equations to KEGG
Added rxnReferences to KEGG
Added rxnConfidenceScores to KEGG
Added metCharges to KEGG
Added geneNames to KEGG
Added geneShortNames to KEGG



Dealing with exchange reactions:


In [31]:
% getting a cell array of metabolites to create exchange reactions for
biomassMets = getBiomassMets(LcrispatusExchangeModel, 'BIOMASS_ps'); % get metabolites involved in the biomass reaction

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 263
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00013'}
    {'C00014'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00036'}
    {'C00037'}
    {'C00041'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00051'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00062'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00083'}
    {'C00084'}
    {'C00085'}
    {'C00089'}
    {'C00092'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00103'}
    {'C00105'}
    {'C00116'}
    {'C00117'}
    {'C00118'}
    {'C00121'}
    {'C00123'}
    {'C00124'}
    {'C00130'}
    {'C00131'}
    {'C00

In [32]:
wasteMets = getCommonWasteMets(LcrispatusExchangeModel); % get list (cell array) of common waste metabolites
essentialMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
communityMets = getCommunityExchangeMets(); % metabolites that are likely to be needed in MCMM community exchanges
allExMets = unique([biomassMets(:); wasteMets(:); essentialMets(:); communityMets(:)], 'stable'); % concatenate while avoiding potential duplicates

% add the exchange reactions to the model
LcrispatusExchangeModel = addExchangeRxnsForModel(LcrispatusExchangeModel, allExMets);

In [33]:
%ensure reversibility in the model
LcrispatusExchangeModel = ensureRevField(LcrispatusExchangeModel);

Model already has rev field

In [34]:
sol3 = solveLP(LcrispatusExchangeModel);
disp(sol3.f);

  33.707865168539328



I have biomass production!!!!


So THIS was the problem! Without exchange reactions, the model wasn't able to function properly!


In [35]:
% pre-export cleaning
LcrispatusExchangeModel = fixSubsystemsField(LcrispatusExchangeModel);
LcrispatusExchangeModel = sanitizeRxnMiriams(LcrispatusExchangeModel);

Sanitized rxnMiriams for 1019 reactions



Now the model is ready for export:


In [36]:
% export the current model version
exportModel(LcrispatusExchangeModel, 'Lcrispatus_draftGEM_v4.xml');

    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [37]:
exportToExcelFormat(LcrispatusExchangeModel, 'Lcrispatus_draftGEM_v4.xlsx');
save('Lcrispatus_draftGEM_v4.mat', 'LcrispatusExchangeModel');


This will be the version used for the MCMM.

## Additional tests

A few more sanity checks and tests prior to moving on to gap\-filling.


In [38]:
% Setting up the draft GEM base
%LcrispatusBiomassDraftModel = importModel('Lcrispatus_draftGEM_v4.xml');
% create a copy to avoid overwriting
LcrispatusBiomassObjDraftModel = LcrispatusExchangeModel;
LcrispatusBiomassObjDraftModel.id = 'LcrispatusDraft';


First, checking for dead\-end metabolites:


In [39]:
deadMets = quickDeadEnds(LcrispatusBiomassObjDraftModel);
fprintf('Dead-end metabolites: %d\n', numel(deadMets));

Dead-end metabolites: 0



Double\-check that the biomass metabolites are indeed present in the model:


In [40]:
biomassMets = getBiomassMets(LcrispatusBiomassObjDraftModel, 'BIOMASS_ps');

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 263
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00013'}
    {'C00014'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00036'}
    {'C00037'}
    {'C00041'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00051'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00062'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00083'}
    {'C00084'}
    {'C00085'}
    {'C00089'}
    {'C00092'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00103'}
    {'C00105'}
    {'C00116'}
    {'C00117'}
    {'C00118'}
    {'C00121'}
    {'C00123'}
    {'C00124'}
    {'C00130'}
    {'C00131'}
    {'C00

In [41]:
missing = setdiff(biomassMets, LcrispatusBiomassObjDraftModel.mets);
disp(missing);


Perform a preliminary test for whether gap\-filling is necessary:


In [42]:
% block all exchange
LcrispatusBiomassObjDraftModel.lb(findExcRxns(LcrispatusBiomassObjDraftModel)) = 0;

% reopen minimal medium
allowMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
for i = 1:numel(allowMets)
    exIdx = find(strcmp(LcrispatusBiomassObjDraftModel.mets, allowMets{i}));
    if ~isempty(exIdx)
        rxnIdx = find(LcrispatusBiomassObjDraftModel.S(exIdx,:));
        LcrispatusBiomassObjDraftModel.lb(rxnIdx) = -1000; % uptake allowed
    end
end

sol4 = solveLP(LcrispatusBiomassObjDraftModel);
disp(sol4.f);

  33.707865168539229



The solution here is the same as before, so this looks good to go.


On Ed's recommendation, I will force the biomass flux to be positive:


In [43]:
% force the biomass flux to be positive
% Set lower and upper bounds of the biomass reaction
rxnID = 'BIOMASS_ps';
biomassIndex = find(ismember(LcrispatusBiomassObjDraftModel.rxns, rxnID));

LcrispatusBiomassObjDraftModel.lb(biomassIndex) = 0.1;  % minimum growth rate
LcrispatusBiomassObjDraftModel.ub(biomassIndex) = 1000; % allow up to 1000


Finally, run FBA on the new model:


In [44]:
sol3 = solveLP(LcrispatusBiomassObjDraftModel);
disp(sol3.f)

  33.707865168539328



Export the model:


In [45]:
exportModel(LcrispatusBiomassObjDraftModel, 'Lcrispatus_draftGEM_v5.xml');

    R00765
    R00839
    R02738
    R04212
    R05086
    R05627
    R12896
    R12958
    R08020
    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [46]:
exportToExcelFormat(LcrispatusBiomassObjDraftModel, 'Lcrispatus_draftGEM_v5.xlsx');
save('Lcrispatus_draftGEM_v5.mat', 'LcrispatusBiomassObjDraftModel');


## Gap\-filling attempt

On the advice of Ed, I will be doing gap\-filling from the whole of the KEGG database. While the model does technically produce biomass and so therefore is viable, it is better to add more exchanges to ensure as high\-quality of an MCMM as possible.


In [47]:
% trying gap filling
[Lcrispatus_filledModel, addedRxns] = fillGaps( ...
    LcrispatusBiomassObjDraftModel, ...
    modelKEGGgr, ...
    false, true, true);

% Check the output of fillGaps
if isempty(Lcrispatus_filledModel)
    disp('fillGaps returned an empty model. Check your input models and required fields.');
elseif iscell(Lcrispatus_filledModel)
    disp(['fillGaps returned a cell array of size: ', mat2str(size(Lcrispatus_filledModel))]);
else
    disp('fillGaps returned a valid model struct.');
end

fillGaps returned an empty model. Check your input models and required fields.



Check the results:


In [48]:
% Did fillGaps return something valid?
class(Lcrispatus_filledModel)

ans = 'cell'

In [49]:
whos Lcrispatus_filledModel

  Name                        Size            Bytes  Class    Attributes
  Lcrispatus_filledModel      0x0                 0  cell



The fact that the filled model is empty implies that fillGaps() didn't find any gaps to fill \- the model should be good as it is.

## Draft *Lactobacillus iners* GEM

Initial GEM creation:


In [50]:
%clear
%clc

% draft with KEGG
%The organisms with KEGG annotation and their associated ids
%(e.g. 'lid' for Lactobacillus iners) can be found from here:
%https://www.kegg.jp/kegg/catalog/org_list.html.
LinersKEGGAnnotation=getKEGGModelForOrganism('lid','','','',1,0,0);

*** The model reconstruction from KEGG based on the annotation available for KEGG Species lid ***
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Pruning the model from non-lid genes... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Constructing GPR associations and annotations for the model... COMPLETE
*** Model reconstruction complete ***

In [51]:
LinersKEGGHomology=getKEGGModelForOrganism('LinersKEGGHMMs','lactobacillus_iners_GCF_011058775-1_prots_edit.fasta','prok90_kegg105','',1,0,0);

*** The model reconstruction from KEGG based on the protein homology search against KEGG Orthology specific HMMs ***
NOTE: Found prok90_kegg105 directory with pre-trained HMMs, it will therefore be used during reconstruction
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Generating the KEGG Orthology specific multi-FASTA files... COMPLETE
Performing clustering and multiple alignment for KEGG Orthology specific protein sets... COMPLETE
Training the KEGG Orthology specific HMMs... COMPLETE
Querying the user-specified FASTA file against the KEGG Orthology specific HMMs... COMPLETE
Parsing the HMM

In [52]:

% merge models
LinersKEGGDraftModel=mergeModels({LinersKEGGAnnotation LinersKEGGHomology});

% check reaction numbers
disp(numel(LinersKEGGHomology.rxns)+numel(LinersKEGGAnnotation.rxns));

        1008

In [53]:
disp(numel(LinersKEGGDraftModel.rxns));

        1008

In [54]:

% further model cleaning
LinersKEGGDraftModel=expandModel(LinersKEGGDraftModel);
LinersKEGGDraftModel=contractModel(LinersKEGGDraftModel);


At this point, a draft GEM has been created for *L. iners*, merging the information from MetaCyc and KEGG. While it needs to be improved, I'll save it for now in Excel format to avoid having to recreate it each time.


In [55]:
exportToExcelFormat(LinersKEGGDraftModel, 'Liners_draftGEM_v1.xlsx');
save('Liners_draftGEM_v1.mat', 'LinersKEGGDraftModel');


## Cleaning *L. iners* draft GEM

In [56]:
%LinersCombinedDraftModel = load('Liners_draftGEM_v1.mat');
LinersCleanDraftModel = cleanGrRulesGenes(LinersKEGGDraftModel);
LinersCleanDraftModel = mergeDuplicateMetNames(LinersCleanDraftModel);
LinersCleanDraftModel = fixSubsystemsField(LinersCleanDraftModel);


The preliminary model should now be ready for export.


In [57]:
exportModel(LinersCleanDraftModel, 'Liners_draftGEM_v2.xml');

    C00856
    C01977
    C02967
    C04157
    C04432
    C17324
    C19722
    C19723
    C20451
    ...and 3 more
Document written

In [58]:
exportToExcelFormat(LinersCleanDraftModel, 'Liners_draftGEM_v2.xlsx');
save('Liners_draftGEM_v2.mat', 'LinersCleanDraftModel');


Saved in SBML standard GEM format.


In [59]:
% Load the model from SBML (.xml)
%LinersCleanDraftModel = importModel('Liners_draftGEM_v2.xml');


Now start quality checks:


In [60]:
%Check out if the newly generated LinersCombinedDraftModel could grow
sol=solveLP(LinersCleanDraftModel);
disp(sol.f);

     0

In [61]:
%The results indicate that this new model is non-functional
% ie., biomass = 0, which means there is no growth

In [62]:
% Look for reactions that produce a "biomass" metabolite
bioMets = find(contains(lower(LinersCleanDraftModel.metNames), 'biomass'));

if isempty(bioMets)
    warning('No metabolites with "biomass" in the name were found.');
else
    % Find reactions involving these metabolites
    [rxnIdx, ~] = find(LinersCleanDraftModel.S(bioMets,:) ~= 0);
    rxnIdx = unique(rxnIdx);
    disp('Potential biomass reactions:');
    disp([rxnIdx, LinersCleanDraftModel.rxns(rxnIdx)', LinersCleanDraftModel.rxnNames(rxnIdx)']);
end


## Adding a biomass reaction

Initial explorations showed that the current preliminary GEM draft doesn't have a biomass reaction. Hence, biomass will always be 0 until this is added. The following code was generated by ChatGPT; it seeds a biomass reaction from common building blocks like amino acids and nucleotides.


In [63]:
[LinersBiomassDraftModel, biomassRxnID] = addPseudoBiomass_debug_all( ...
    LinersCleanDraftModel, ...
    'BIOMASS_ps', ...
    [], ...
    true, ...
    'Liners_biomass_debug.txt', ...
    'Liners_biomass_metabolites.csv' ...
);

Precursor "L-alanine" → 4 matches found
    Match: "UDP-N-acetylmuramate:L-alanine ligase (ADP-forming)" (ID: R03193)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "UDP-N-acetylmuramate" (ID: C01050)
    Match: "UDP-N-acetylmuramoyl-L-alanine:D-glutamate ligase(ADP-forming)" (ID: R02783)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "D-Glutamate" (ID: C00217)
        Metabolite: "UDP-N-acetylmuramoyl-L-alanine" (ID: C01212)
    Match: "L-Alanine:tRNA(Ala) ligase (AMP-forming)" (ID: R03038)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "tRNA(Ala)" (ID: C01635)
    Match: "L-selenocysteine selenide-lyase (L-alanine-forming)" (ID: R03599)
        Metabolite: "Reduced acceptor" (ID: C00030)
        Metabolite: "L-Selenocysteine" (ID: C05688)
Precursor "L-arginine" → 1 matches found
    Match: "L-Arginine:tRNA(Arg) ligase (AMP-forming)" (ID: R03646)
        Metabo



Quick cleaning to ensure RAVEN structures are correctly kept:


In [64]:
LinersBiomassDraftModel = syncModelStruct(LinersBiomassDraftModel, 'Liners_biomass_sync_log.csv');

syncModelStruct: Log saved to "Liners_biomass_sync_log.csv"

In [65]:
checkModelCoreConsistency(LinersBiomassDraftModel);

--- BASIC DIMENSION CHECKS ---
Reactions: 574
Metabolites: 825
Genes: 437
Empty grRules in 145 reactions

In [66]:
LinersBiomassDraftModel = sanitizeGEMforSBML(LinersBiomassDraftModel);
LinersBiomassDraftModel = fixRxnConfidenceScores(LinersBiomassDraftModel);


I will now save the new model with the biomass reaction included:


In [67]:
exportModel(LinersBiomassDraftModel, 'Liners_draftGEM_v3.xml');

Document written

In [68]:
exportToExcelFormat(LinersBiomassDraftModel, 'Liners_draftGEM_v3.xlsx');
save('Liners_draftGEM_v3.mat', 'LinersBiomassDraftModel');


Now that a biomass reaction has been added to the model, I will try to solve again:


In [69]:
%LinersBiomassDraftModel = importModel('Liners_draftGEM_v3.xml');
sol2 = solveLP(LinersBiomassDraftModel);
disp(sol2.f)

     0



Still 0, so still no growth.

## Addition of exchange reactions

The simplified gap report:


In [70]:
% taking a look at the gap situation
%gapReport(LinersBiomassDraftModel);
gapReport_vSimple(LinersBiomassDraftModel);

Dead-end metabolites: 461
    {'C00018'}
    {'C00026'}
    {'C00032'}
    {'C00036'}
    {'C00040'}
    {'C00053'}
    {'C00071'}
    {'C00083'}
    {'C00088'}
    {'C00094'}
    {'C00104'}
    {'C00127'}
    {'C00132'}
    {'C00140'}
    {'C00155'}
    {'C00159'}
    {'C00170'}
    {'C00184'}
    {'C00201'}
    {'C00224'}
    {'C00233'}
    {'C00237'}
    {'C00253'}
    {'C00262'}
    {'C00263'}
    {'C00310'}
    {'C00353'}
    {'C00363'}
    {'C00378'}
    {'C00395'}
    {'C00402'}
    {'C00447'}
    {'C00450'}
    {'C00454'}
    {'C00458'}
    {'C00577'}
    {'C00620'}
    {'C00627'}
    {'C00647'}
    {'C00653'}
    {'C00680'}
    {'C00700'}
    {'C00718'}
    {'C00721'}
    {'C00842'}
    {'C00957'}
    {'C00973'}
    {'C01019'}
    {'C01024'}
    {'C01041'}
    {'C01096'}
    {'C01133'}
    {'C01252'}
    {'C01297'}
    {'C01345'}
    {'C01419'}
    {'C01424'}
    {'C01451'}
    {'C01471'}
    {'C01592'}
    {'C01637'}
    {'C01642'}
    {'C01717'}
    {'C01752'}
    {'C01931'}



In order for this to be a functional GEM, exchange reactions need to be added.


In [71]:
% Setting up the draft GEM base
%LinersBiomassDraftModel = importModel('Liners_draftGEM_v3.xml');
% create a copy to avoid overwriting
LinersExchangeModel = LinersBiomassDraftModel;
LinersExchangeModel.id = 'LinersDraft';


Perform some cleaning:


In [72]:
% deal with grRules issues
LinersExchangeModel = ensureGrRules(LinersExchangeModel);
modelKEGGgr = ensureGrRules(modelKEGG);

Patched grRules to match 11588 rxns

In [73]:
% further sanitization
modelKEGGgr = padMissingFields(modelKEGGgr, LinersExchangeModel);

Added geneMiriams to KEGG
Added rxnFrom to KEGG
Added metFrom to KEGG
Added geneFrom to KEGG
Added rules to KEGG
Added equations to KEGG
Added rxnReferences to KEGG
Added rxnConfidenceScores to KEGG
Added metCharges to KEGG
Added geneNames to KEGG
Added geneShortNames to KEGG



Dealing with exchange reactions:


In [74]:
% getting a cell array of metabolites to create exchange reactions for
biomassMets = getBiomassMets(LinersExchangeModel, 'BIOMASS_ps'); % get metabolites involved in the biomass reaction

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 202
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00013'}
    {'C00014'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00030'}
    {'C00033'}
    {'C00041'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00084'}
    {'C00085'}
    {'C00092'}
    {'C00093'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00103'}
    {'C00105'}
    {'C00117'}
    {'C00118'}
    {'C00123'}
    {'C00130'}
    {'C00131'}
    {'C00136'}
    {'C00143'}
    {'C00144'}
    {'C00148'}
    {'C00163'}
    {'C00169'}
    {'C00183'}
    {'C00185'}
    {'C00186'}
    {'C00188'}
    {'C00197'}
    {'C00214'}
    {'C00

In [75]:
wasteMets = getCommonWasteMets(LinersExchangeModel); % get list (cell array) of common waste metabolites
essentialMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
communityMets = getCommunityExchangeMets(); % metabolites that are likely to be needed in MCMM community exchanges
allExMets = unique([biomassMets(:); wasteMets(:); essentialMets(:); communityMets(:)], 'stable'); % concatenate while avoiding potential duplicates

% add the exchange reactions to the model
LinersExchangeModel = addExchangeRxnsForModel(LinersExchangeModel, allExMets);

In [76]:

% ensure reversibility of reactions
LinersExchangeModel = ensureRevField(LinersExchangeModel);

Model already has rev field

In [77]:
sol3 = solveLP(LinersExchangeModel);
disp(sol3.f);

  66.666666666666671



There is now biomass production!


In [78]:
% pre-export cleaning
LinersExchangeModel = fixSubsystemsField(LinersExchangeModel);
LinersExchangeModel = sanitizeRxnMiriams(LinersExchangeModel);

Sanitized rxnMiriams for 781 reactions



Now the model is ready for export:


In [79]:
% export the current model version
exportModel(LinersExchangeModel, 'Liners_draftGEM_v4.xml');

Document written

In [80]:
exportToExcelFormat(LinersExchangeModel, 'Liners_draftGEM_v4.xlsx');
save('Liners_draftGEM_v4.mat', 'LinersExchangeModel');


This will be the version used for the MCMM.

## Additional tests

A few more sanity checks and tests prior to moving on to gap\-filling.


In [81]:
% Setting up the draft GEM base
%LinersBiomassDraftModel = importModel('Liners_draftGEM_v4.xml');
% create a copy to avoid overwriting
LinersBiomassObjDraftModel = LinersExchangeModel;
LinersBiomassObjDraftModel.id = 'LinersDraft';


First, checking for dead\-end metabolites:


In [82]:
deadMets = quickDeadEnds(LinersBiomassObjDraftModel);
fprintf('Dead-end metabolites: %d\n', numel(deadMets));

Dead-end metabolites: 0



Double\-check that the biomass metabolites are indeed present in the model:


In [83]:
biomassMets = getBiomassMets(LinersBiomassObjDraftModel, 'BIOMASS_ps');

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 202
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00013'}
    {'C00014'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00030'}
    {'C00033'}
    {'C00041'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00084'}
    {'C00085'}
    {'C00092'}
    {'C00093'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00103'}
    {'C00105'}
    {'C00117'}
    {'C00118'}
    {'C00123'}
    {'C00130'}
    {'C00131'}
    {'C00136'}
    {'C00143'}
    {'C00144'}
    {'C00148'}
    {'C00163'}
    {'C00169'}
    {'C00183'}
    {'C00185'}
    {'C00186'}
    {'C00188'}
    {'C00197'}
    {'C00214'}
    {'C00

In [84]:
missing = setdiff(biomassMets, LinersBiomassObjDraftModel.mets);
disp(missing);


Perform a preliminary test for whether gap\-filling is necessary:


In [85]:
% block all exchange
LinersBiomassObjDraftModel.lb(findExcRxns(LinersBiomassObjDraftModel)) = 0;

% reopen minimal medium
allowMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
for i = 1:numel(allowMets)
    exIdx = find(strcmp(LinersBiomassObjDraftModel.mets, allowMets{i}));
    if ~isempty(exIdx)
        rxnIdx = find(LinersBiomassObjDraftModel.S(exIdx,:));
        LinersBiomassObjDraftModel.lb(rxnIdx) = -1000; % uptake allowed
    end
end

sol4 = solveLP(LinersBiomassObjDraftModel);
disp(sol4.f);

  53.333333333333258



The solution here is the same as before, so this looks good to go.


On Ed's recommendation, I will force the biomass flux to be positive:


In [86]:
% force the biomass flux to be positive
% Set lower and upper bounds of the biomass reaction
rxnID = 'BIOMASS_ps';
biomassIndex = find(ismember(LinersBiomassObjDraftModel.rxns, rxnID));

LinersBiomassObjDraftModel.lb(biomassIndex) = 0.1;  % minimum growth rate
LinersBiomassObjDraftModel.ub(biomassIndex) = 1000; % allow up to 1000


Finally, run FBA on the new model:


In [87]:
sol3 = solveLP(LinersBiomassObjDraftModel);
disp(sol3.f)

  53.333333333333357



Export the model:


In [88]:
exportModel(LinersBiomassObjDraftModel, 'Liners_draftGEM_v5.xml');

    R00765
    R02738
    R04212
    R05627
    R11037
    R12896
    R08020
    R12958
Document written

In [89]:
exportToExcelFormat(LinersBiomassObjDraftModel, 'Liners_draftGEM_v5.xlsx');
save('Liners_draftGEM_v5.mat', 'LinersBiomassObjDraftModel');


## Gap\-filling attempt

On the advice of Ed, I will be doing gap\-filling from the whole of the KEGG database. While the model does technically produce biomass and so therefore is viable, it is better to add more exchanges to ensure as high\-quality of an MCMM as possible.


In [90]:
% trying gap filling
[Liners_filledModel, addedRxns] = fillGaps( ...
    LinersBiomassObjDraftModel, ...
    modelKEGGgr, ...
    false, true, true);

% Check the output of fillGaps
if isempty(Liners_filledModel)
    disp('fillGaps returned an empty model. Check your input models and required fields.');
elseif iscell(Liners_filledModel)
    disp(['fillGaps returned a cell array of size: ', mat2str(size(Liners_filledModel))]);
else
    disp('fillGaps returned a valid model struct.');
end

fillGaps returned an empty model. Check your input models and required fields.



Check the results:


In [91]:
% Did fillGaps return something valid?
class(Liners_filledModel)

ans = 'cell'

In [92]:
whos Liners_filledModel

  Name                    Size            Bytes  Class    Attributes
  Liners_filledModel      0x0                 0  cell



As before, it appears that gap\-filling is unnecessary in this case.

## Draft *Gardnerella vaginalis* GEM

Initial GEM creation:


In [93]:
%clear
%clc

% draft with KEGG
%The organisms with KEGG annotation and their associated ids
%(e.g. 'gvh' for Gardnerella vaginalis HMP9231) can be found from here:
%https://www.kegg.jp/kegg/catalog/org_list.html.
GvaginalisKEGGAnnotation=getKEGGModelForOrganism('gvh','','','',1,0,0);

*** The model reconstruction from KEGG based on the annotation available for KEGG Species gvh ***
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Pruning the model from non-gvh genes... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Constructing GPR associations and annotations for the model... COMPLETE
*** Model reconstruction complete ***

In [94]:
GvaginalisKEGGHomology=getKEGGModelForOrganism('GvaginalisKEGGHMMs','gardnerella_vaginalis_GCF_001042655-1_prots_edit.fasta','prok90_kegg105','',1,0,0);

*** The model reconstruction from KEGG based on the protein homology search against KEGG Orthology specific HMMs ***
NOTE: Found prok90_kegg105 directory with pre-trained HMMs, it will therefore be used during reconstruction
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Generating the KEGG Orthology specific multi-FASTA files... COMPLETE
Performing clustering and multiple alignment for KEGG Orthology specific protein sets... COMPLETE
Training the KEGG Orthology specific HMMs... COMPLETE
Querying the user-specified FASTA file against the KEGG Orthology specific HMMs... COMPLETE
Parsing the HMM

In [95]:

% merge models
GvaginalisKEGGDraftModel=mergeModels({GvaginalisKEGGAnnotation GvaginalisKEGGHomology});

% check reaction numbers
disp(numel(GvaginalisKEGGHomology.rxns)+numel(GvaginalisKEGGAnnotation.rxns));

        1283

In [96]:
disp(numel(GvaginalisKEGGDraftModel.rxns));

        1283

In [97]:

% further model cleaning
GvaginalisKEGGDraftModel=expandModel(GvaginalisKEGGDraftModel);
GvaginalisKEGGDraftModel=contractModel(GvaginalisKEGGDraftModel);


At this point, a draft GEM has been created for *G. vaginalis*, merging the information from MetaCyc and KEGG. While it needs to be improved, I'll save it for now in Excel format to avoid having to recreate it each time.


In [98]:
exportToExcelFormat(GvaginalisKEGGDraftModel, 'Gvaginalis_draftGEM_v1.xlsx');
save('Gvaginalis_draftGEM_v1.mat', 'GvaginalisKEGGDraftModel');


## Cleaning *G. vaginalis* draft GEM

In [99]:
%GvaginalisCombinedDraftModel = load('Gvaginalis_draftGEM_v1.mat');
GvaginalisCleanDraftModel = cleanGrRulesGenes(GvaginalisKEGGDraftModel);
GvaginalisCleanDraftModel = mergeDuplicateMetNames(GvaginalisCleanDraftModel);
GvaginalisCleanDraftModel = fixSubsystemsField(GvaginalisCleanDraftModel);


The preliminary model should now be ready for export.


In [100]:
exportModel(GvaginalisCleanDraftModel, 'Gvaginalis_draftGEM_v2.xml');

    C00856
    C01977
    C02967
    C04157
    C04432
    C17324
    C19722
    C19723
    C20446
    ...and 4 more
    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [101]:
exportToExcelFormat(GvaginalisCleanDraftModel, 'Gvaginalis_draftGEM_v2.xlsx');
save('Gvaginalis_draftGEM_v2.mat', 'GvaginalisCleanDraftModel');


Saved in SBML standard GEM format.


In [102]:
% Load the model from SBML (.xml)
%GvaginalisCleanDraftModel = importModel('Gvaginalis_draftGEM_v2.xml');


Now start quality checks:


In [103]:
%Check out if the newly generated GvaginalisCombinedDraftModel could grow
sol=solveLP(GvaginalisCleanDraftModel);
disp(sol.f);

     0

In [104]:
%The results indicate that this new model is non-functional
% ie., biomass = 0, which means there is no growth

In [105]:
% Look for reactions that produce a "biomass" metabolite
bioMets = find(contains(lower(GvaginalisCleanDraftModel.metNames), 'biomass'));

if isempty(bioMets)
    warning('No metabolites with "biomass" in the name were found.');
else
    % Find reactions involving these metabolites
    [rxnIdx, ~] = find(GvaginalisCleanDraftModel.S(bioMets,:) ~= 0);
    rxnIdx = unique(rxnIdx);
    disp('Potential biomass reactions:');
    disp([rxnIdx, GvaginalisCleanDraftModel.rxns(rxnIdx)', GvaginalisCleanDraftModel.rxnNames(rxnIdx)']);
end


## Adding a biomass reaction

Initial explorations showed that the current preliminary GEM draft doesn't have a biomass reaction. Hence, biomass will always be 0 until this is added. The following code was generated by ChatGPT; it seeds a biomass reaction from common building blocks like amino acids and nucleotides.


In [106]:
[GvaginalisBiomassDraftModel, biomassRxnID] = addPseudoBiomass_debug_all( ...
    GvaginalisCleanDraftModel, ...
    'BIOMASS_ps', ...
    [], ...
    true, ...
    'Gvaginalis_biomass_debug.txt', ...
    'Gvaginalis_biomass_metabolites.csv' ...
);

Precursor "L-alanine" → 6 matches found
    Match: "L-Alanine:2-oxoglutarate aminotransferase" (ID: R00258)
        Metabolite: "2-Oxoglutarate" (ID: C00026)
        Metabolite: "L-Alanine" (ID: C00041)
    Match: "3-sulfino-L-alanine:2-oxoglutarate aminotransferase" (ID: R02619)
        Metabolite: "2-Oxoglutarate" (ID: C00026)
        Metabolite: "3-Sulfino-L-alanine" (ID: C00606)
    Match: "UDP-N-acetylmuramoyl-L-alanine:D-glutamate ligase(ADP-forming)" (ID: R02783)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "D-Glutamate" (ID: C00217)
        Metabolite: "UDP-N-acetylmuramoyl-L-alanine" (ID: C01212)
    Match: "L-Alanine:tRNA(Ala) ligase (AMP-forming)" (ID: R03038)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "tRNA(Ala)" (ID: C01635)
    Match: "UDP-N-acetylmuramate:L-alanine ligase (ADP-forming)" (ID: R03193)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabol



Quick cleaning to ensure RAVEN structures are correctly kept:


In [107]:
GvaginalisBiomassDraftModel = syncModelStruct(GvaginalisBiomassDraftModel, 'Gvaginalis_biomass_sync_log.csv');

syncModelStruct: Log saved to "Gvaginalis_biomass_sync_log.csv"

In [108]:
checkModelCoreConsistency(GvaginalisBiomassDraftModel);

--- BASIC DIMENSION CHECKS ---
Reactions: 706
Metabolites: 979
Genes: 583
Empty grRules in 147 reactions

In [109]:
GvaginalisBiomassDraftModel = sanitizeGEMforSBML(GvaginalisBiomassDraftModel);
GvaginalisBiomassDraftModel = fixRxnConfidenceScores(GvaginalisBiomassDraftModel);


I will now save the new model with the biomass reaction included:


In [110]:
exportModel(GvaginalisBiomassDraftModel, 'Gvaginalis_draftGEM_v3.xml');

    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [111]:
exportToExcelFormat(GvaginalisBiomassDraftModel, 'Gvaginalis_draftGEM_v3.xlsx');
save('Gvaginalis_draftGEM_v3.mat', 'GvaginalisBiomassDraftModel');


Now that a biomass reaction has been added to the model, I will try to solve again:


In [112]:
%GvaginalisBiomassDraftModel = importModel('Gvaginalis_draftGEM_v3.xml');
sol2 = solveLP(GvaginalisBiomassDraftModel);
disp(sol2.f)

     0



Still 0, so still no growth.

## Addition of exchange reactions

The simplified gap report:


In [113]:
% taking a look at the gap situation
%gapReport(GvaginalisBiomassDraftModel);
gapReport_vSimple(GvaginalisBiomassDraftModel);

Dead-end metabolites: 494
    {'C00016'}
    {'C00040'}
    {'C00042'}
    {'C00071'}
    {'C00081'}
    {'C00088'}
    {'C00089'}
    {'C00127'}
    {'C00132'}
    {'C00173'}
    {'C00201'}
    {'C00232'}
    {'C00243'}
    {'C00250'}
    {'C00252'}
    {'C00253'}
    {'C00259'}
    {'C00299'}
    {'C00302'}
    {'C00330'}
    {'C00353'}
    {'C00354'}
    {'C00363'}
    {'C00376'}
    {'C00447'}
    {'C00450'}
    {'C00454'}
    {'C00506'}
    {'C00536'}
    {'C00577'}
    {'C00667'}
    {'C00671'}
    {'C00683'}
    {'C00700'}
    {'C00721'}
    {'C00935'}
    {'C00962'}
    {'C00966'}
    {'C00973'}
    {'C00988'}
    {'C01019'}
    {'C01024'}
    {'C01041'}
    {'C01094'}
    {'C01100'}
    {'C01101'}
    {'C01117'}
    {'C01133'}
    {'C01177'}
    {'C01190'}
    {'C01252'}
    {'C01267'}
    {'C01289'}
    {'C01290'}
    {'C01297'}
    {'C01345'}
    {'C01353'}
    {'C01390'}
    {'C01419'}
    {'C01424'}
    {'C01471'}
    {'C01592'}
    {'C01642'}
    {'C01717'}
    {'C01720'}



In order for this to be a functional GEM, exchange reactions need to be added.


In [114]:
% Setting up the draft GEM base
%GvaginalisBiomassDraftModel = importModel('Gvaginalis_draftGEM_v3.xml');
% create a copy to avoid overwriting
GvaginalisExchangeModel = GvaginalisBiomassDraftModel;
GvaginalisExchangeModel.id = 'GvaginalisDraft';


Perform some cleaning:


In [115]:
% deal with grRules issues
GvaginalisExchangeModel = ensureGrRules(GvaginalisExchangeModel);
modelKEGGgr = ensureGrRules(modelKEGG);

Patched grRules to match 11588 rxns

In [116]:
% further sanitization
modelKEGGgr = padMissingFields(modelKEGGgr, GvaginalisExchangeModel);

Added geneMiriams to KEGG
Added rxnFrom to KEGG
Added metFrom to KEGG
Added geneFrom to KEGG
Added rules to KEGG
Added equations to KEGG
Added rxnReferences to KEGG
Added rxnConfidenceScores to KEGG
Added metCharges to KEGG
Added geneNames to KEGG
Added geneShortNames to KEGG



Dealing with exchange reactions:


In [117]:
% getting a cell array of metabolites to create exchange reactions for
biomassMets = getBiomassMets(GvaginalisExchangeModel, 'BIOMASS_ps'); % get metabolites involved in the biomass reaction

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 273
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00013'}
    {'C00014'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00037'}
    {'C00041'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00062'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00083'}
    {'C00084'}
    {'C00085'}
    {'C00092'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00103'}
    {'C00105'}
    {'C00109'}
    {'C00117'}
    {'C00118'}
    {'C00120'}
    {'C00121'}
    {'C00123'}
    {'C00130'}
    {'C00131'}
    {'C00135'}
    {'C00136'}
    {'C00143'}
    {'C00

In [118]:
wasteMets = getCommonWasteMets(GvaginalisExchangeModel); % get list (cell array) of common waste metabolites
essentialMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
communityMets = getCommunityExchangeMets(); % metabolites that are likely to be needed in MCMM community exchanges
allExMets = unique([biomassMets(:); wasteMets(:); essentialMets(:); communityMets(:)], 'stable'); % concatenate while avoiding potential duplicates

% add the exchange reactions to the model
GvaginalisExchangeModel = addExchangeRxnsForModel(GvaginalisExchangeModel, allExMets);

In [119]:

% ensure reversibility of reactions
GvaginalisExchangeModel = ensureRevField(GvaginalisExchangeModel);

Model already has rev field

In [120]:
sol3 = solveLP(GvaginalisExchangeModel);
disp(sol3.f);

  36.585365853658530



There is now biomass production!


In [121]:
% pre-export cleaning
GvaginalisExchangeModel = fixSubsystemsField(GvaginalisExchangeModel);
GvaginalisExchangeModel = sanitizeRxnMiriams(GvaginalisExchangeModel);

Sanitized rxnMiriams for 984 reactions



Now the model is ready for export:


In [122]:
% export the current model version
exportModel(GvaginalisExchangeModel, 'Gvaginalis_draftGEM_v4.xml');

    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [123]:
exportToExcelFormat(GvaginalisExchangeModel, 'Gvaginalis_draftGEM_v4.xlsx');
save('Gvaginalis_draftGEM_v4.mat', 'GvaginalisExchangeModel');


This will be the version used for the MCMM.

## Additional tests

A few more sanity checks and tests prior to moving on to gap\-filling.


In [124]:
% Setting up the draft GEM base
%GvaginalisBiomassDraftModel = importModel('Gvaginalis_draftGEM_v4.xml');
% create a copy to avoid overwriting
GvaginalisBiomassObjDraftModel = GvaginalisExchangeModel;
GvaginalisBiomassObjDraftModel.id = 'GvaginalisDraft';


First, checking for dead\-end metabolites:


In [125]:
deadMets = quickDeadEnds(GvaginalisBiomassObjDraftModel);
fprintf('Dead-end metabolites: %d\n', numel(deadMets));

Dead-end metabolites: 0



Double\-check that the biomass metabolites are indeed present in the model:


In [126]:
biomassMets = getBiomassMets(GvaginalisBiomassObjDraftModel, 'BIOMASS_ps');

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 273
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00013'}
    {'C00014'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00037'}
    {'C00041'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00062'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00083'}
    {'C00084'}
    {'C00085'}
    {'C00092'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00103'}
    {'C00105'}
    {'C00109'}
    {'C00117'}
    {'C00118'}
    {'C00120'}
    {'C00121'}
    {'C00123'}
    {'C00130'}
    {'C00131'}
    {'C00135'}
    {'C00136'}
    {'C00143'}
    {'C00

In [127]:
missing = setdiff(biomassMets, GvaginalisBiomassObjDraftModel.mets);
disp(missing);


Perform a preliminary test for whether gap\-filling is necessary:


In [128]:
% block all exchange
GvaginalisBiomassObjDraftModel.lb(findExcRxns(GvaginalisBiomassObjDraftModel)) = 0;

% reopen minimal medium
allowMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
for i = 1:numel(allowMets)
    exIdx = find(strcmp(GvaginalisBiomassObjDraftModel.mets, allowMets{i}));
    if ~isempty(exIdx)
        rxnIdx = find(GvaginalisBiomassObjDraftModel.S(exIdx,:));
        GvaginalisBiomassObjDraftModel.lb(rxnIdx) = -1000; % uptake allowed
    end
end

sol4 = solveLP(GvaginalisBiomassObjDraftModel);
disp(sol4.f);

  36.585365853658459



The solution here is the same as before, so this looks good to go.


On Ed's recommendation, I will force the biomass flux to be positive:


In [129]:
% force the biomass flux to be positive
% Set lower and upper bounds of the biomass reaction
rxnID = 'BIOMASS_ps';
biomassIndex = find(ismember(GvaginalisBiomassObjDraftModel.rxns, rxnID));

GvaginalisBiomassObjDraftModel.lb(biomassIndex) = 0.1;  % minimum growth rate
GvaginalisBiomassObjDraftModel.ub(biomassIndex) = 1000; % allow up to 1000


Finally, run FBA on the new model:


In [130]:
sol3 = solveLP(GvaginalisBiomassObjDraftModel);
disp(sol3.f)

  36.585365853658537



Export the model:


In [131]:
exportModel(GvaginalisBiomassObjDraftModel, 'Gvaginalis_draftGEM_v5.xml');

    R00765
    R04212
    R05086
    R08020
    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [132]:
exportToExcelFormat(GvaginalisBiomassObjDraftModel, 'Gvaginalis_draftGEM_v5.xlsx');
save('Gvaginalis_draftGEM_v5.mat', 'GvaginalisBiomassObjDraftModel');


## Gap\-filling attempt

On the advice of Ed, I will be doing gap\-filling from the whole of the KEGG database. While the model does technically produce biomass and so therefore is viable, it is better to add more exchanges to ensure as high\-quality of an MCMM as possible.


In [133]:
% trying gap filling
[Gvaginalis_filledModel, addedRxns] = fillGaps( ...
    GvaginalisBiomassObjDraftModel, ...
    modelKEGGgr, ...
    false, true, true);

% Check the output of fillGaps
if isempty(Gvaginalis_filledModel)
    disp('fillGaps returned an empty model. Check your input models and required fields.');
elseif iscell(Gvaginalis_filledModel)
    disp(['fillGaps returned a cell array of size: ', mat2str(size(Gvaginalis_filledModel))]);
else
    disp('fillGaps returned a valid model struct.');
end

fillGaps returned an empty model. Check your input models and required fields.



Check the results:


In [134]:
% Did fillGaps return something valid?
class(Gvaginalis_filledModel)

ans = 'cell'

In [135]:
whos Gvaginalis_filledModel

  Name                        Size            Bytes  Class    Attributes
  Gvaginalis_filledModel      0x0                 0  cell



As before, it appears that gap\-filling is unnecessary in this case.

## Draft *Prevotella bivia* GEM

Initial GEM creation:


In [136]:
%clear
%clc

% draft with KEGG
%The organisms with KEGG annotation and their associated ids
%(e.g. 'pbiv' for Prevotella bivia) can be found from here:
%https://www.kegg.jp/kegg/catalog/org_list.html.
%PbiviaKEGGAnnotation=getKEGGModelForOrganism('pbiv','','','',1,0,0);
% in this case, things were more complicated
% pbiv was added to KEGG in 2023
% but appears not to be included in the version of RAVEN that is available
% in MATLAB
% so I'm using the closest relative I can find that's available
% it looks like p. denticola seems to cluster with P. bivia genomically
% ref: https://www.sciencedirect.com/science/article/pii/S2213716523001789
% (Peng et al. 2023)
% in KEGG, P. denticola is 'pdn'
PdenticolaKEGGAnnotation=getKEGGModelForOrganism('pdn','','','',1,0,0);

*** The model reconstruction from KEGG based on the annotation available for KEGG Species pdn ***
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Pruning the model from non-pdn genes... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Constructing GPR associations and annotations for the model... COMPLETE
*** Model reconstruction complete ***

In [137]:
% local FASTA model
PbiviaKEGGHomology=getKEGGModelForOrganism('PbiviaKEGGHMMs','prevotella_bivia_GCF_000262545-1_prots_edit.fasta','prok90_kegg105','',1,0,0);

*** The model reconstruction from KEGG based on the protein homology search against KEGG Orthology specific HMMs ***
NOTE: Found prok90_kegg105 directory with pre-trained HMMs, it will therefore be used during reconstruction
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Generating the KEGG Orthology specific multi-FASTA files... COMPLETE
Performing clustering and multiple alignment for KEGG Orthology specific protein sets... COMPLETE
Training the KEGG Orthology specific HMMs... COMPLETE
Querying the user-specified FASTA file against the KEGG Orthology specific HMMs... COMPLETE
Parsing the HMM

In [138]:

% merge models
PbiviaKEGGDraftModel=mergeModels({PdenticolaKEGGAnnotation PbiviaKEGGHomology});

% check reaction numbers
disp(numel(PbiviaKEGGHomology.rxns)+numel(PdenticolaKEGGAnnotation.rxns));

        1623

In [139]:
disp(numel(PbiviaKEGGDraftModel.rxns));

        1623

In [140]:

% further model cleaning
PbiviaKEGGDraftModel=expandModel(PbiviaKEGGDraftModel);
PbiviaKEGGDraftModel=contractModel(PbiviaKEGGDraftModel);


At this point, a draft GEM has been created for *P. bivia*, merging the information from MetaCyc and KEGG. While it needs to be improved, I'll save it for now in Excel format to avoid having to recreate it each time.


In [141]:
exportToExcelFormat(PbiviaKEGGDraftModel, 'Pbivia_draftGEM_v1.xlsx');
save('Pbivia_draftGEM_v1.mat', 'PbiviaKEGGDraftModel');


## Cleaning *P. bivia* draft GEM

In [142]:
%PbiviaCombinedDraftModel = load('Pbivia_draftGEM_v1.mat');
PbiviaCleanDraftModel = cleanGrRulesGenes(PbiviaKEGGDraftModel);
PbiviaCleanDraftModel = mergeDuplicateMetNames(PbiviaCleanDraftModel);
PbiviaCleanDraftModel = mergeDuplicateMetNames2(PbiviaCleanDraftModel);
PbiviaCleanDraftModel = fixSubsystemsField(PbiviaCleanDraftModel);


The preliminary model should now be ready for export.


In [143]:
exportModel(PbiviaCleanDraftModel, 'Pbivia_draftGEM_v2.xml');

    C01977
    C01978
    C04157
    C04432
    C17324
    C19647
    C19722
    C19723
    C20446
    ...and 9 more
    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [144]:
exportToExcelFormat(PbiviaCleanDraftModel, 'Pbivia_draftGEM_v2.xlsx');
save('Pbivia_draftGEM_v2.mat', 'PbiviaCleanDraftModel');


Saved in SBML standard GEM format.


In [145]:
% Load the model from SBML (.xml)
%PbiviaCleanDraftModel = importModel('Pbivia_draftGEM_v2.xml');


Now start quality checks:


In [146]:
%Check out if the newly generated PbiviaCombinedDraftModel could grow
sol=solveLP(PbiviaCleanDraftModel);
disp(sol.f);

     0

In [147]:
%The results indicate that this new model is non-functional
% ie., biomass = 0, which means there is no growth

In [148]:
% Look for reactions that produce a "biomass" metabolite
bioMets = find(contains(lower(PbiviaCleanDraftModel.metNames), 'biomass'));

if isempty(bioMets)
    warning('No metabolites with "biomass" in the name were found.');
else
    % Find reactions involving these metabolites
    [rxnIdx, ~] = find(PbiviaCleanDraftModel.S(bioMets,:) ~= 0);
    rxnIdx = unique(rxnIdx);
    disp('Potential biomass reactions:');
    disp([rxnIdx, PbiviaCleanDraftModel.rxns(rxnIdx)', PbiviaCleanDraftModel.rxnNames(rxnIdx)']);
end


## Adding a biomass reaction

Initial explorations showed that the current preliminary GEM draft doesn't have a biomass reaction. Hence, biomass will always be 0 until this is added. The following code was generated by ChatGPT; it seeds a biomass reaction from common building blocks like amino acids and nucleotides.


In [149]:
[PbiviaBiomassDraftModel, biomassRxnID] = addPseudoBiomass_gramNegative( ...
    PbiviaCleanDraftModel, ...
    'BIOMASS_ps', ...
    [], ...
    true, ...
    'Pbivia_biomass_debug.txt', ...
    'Pbivia_biomass_metabolites.csv' ...
);

Precursor "L-alanine" → 7 matches found
    Match: "3-sulfino-L-alanine:2-oxoglutarate aminotransferase" (ID: R02619_EXP_1)
        Metabolite: "2-Oxoglutarate" (ID: C00026)
        Metabolite: "3-Sulfino-L-alanine" (ID: C00606)
    Match: "UDP-N-acetylmuramoyl-L-alanine:D-glutamate ligase(ADP-forming)" (ID: R02783)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "D-Glutamate" (ID: C00217)
        Metabolite: "UDP-N-acetylmuramoyl-L-alanine" (ID: C01212)
    Match: "L-Alanine:tRNA(Ala) ligase (AMP-forming)" (ID: R03038)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "tRNA(Ala)" (ID: C01635)
    Match: "UDP-N-acetylmuramate:L-alanine ligase (ADP-forming)" (ID: R03193)
        Metabolite: "ATP" (ID: C00002)
        Metabolite: "L-Alanine" (ID: C00041)
        Metabolite: "UDP-N-acetylmuramate" (ID: C01050)
    Match: "pimeloyl-CoA:L-alanine C-carboxyhexanoyltransferase (decarboxylating)" (ID: R03210)
        Metabolite: 



Quick cleaning to ensure RAVEN structures are correctly kept:


In [150]:
PbiviaBiomassDraftModel = syncModelStruct(PbiviaBiomassDraftModel, 'Pbivia_biomass_sync_log.csv');

syncModelStruct: Log saved to "Pbivia_biomass_sync_log.csv"

In [151]:
checkModelCoreConsistency(PbiviaBiomassDraftModel);

--- BASIC DIMENSION CHECKS ---
Reactions: 938
Metabolites: 1159
Genes: 809
Empty grRules in 142 reactions

In [152]:
PbiviaBiomassDraftModel = sanitizeGEMforSBML(PbiviaBiomassDraftModel);
PbiviaBiomassDraftModel = fixRxnConfidenceScores(PbiviaBiomassDraftModel);


I will now save the new model with the biomass reaction included:


In [153]:
exportModel(PbiviaBiomassDraftModel, 'Pbivia_draftGEM_v3.xml');

    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [154]:
exportToExcelFormat(PbiviaBiomassDraftModel, 'Pbivia_draftGEM_v3.xlsx');
save('Pbivia_draftGEM_v3.mat', 'PbiviaBiomassDraftModel');


Now that a biomass reaction has been added to the model, I will try to solve again:


In [155]:
%PbiviaBiomassDraftModel = importModel('Pbivia_draftGEM_v3.xml');
sol2 = solveLP(PbiviaBiomassDraftModel);
disp(sol2.f)

     0



Still 0, so still no growth.

## Addition of exchange reactions

The simplified gap report:


In [156]:
% taking a look at the gap situation
%gapReport(PbiviaBiomassDraftModel);
gapReport_vSimple(PbiviaBiomassDraftModel);

Dead-end metabolites: 559
    {'C00040'}
    {'C00052'}
    {'C00071'}
    {'C00084'}
    {'C00104'}
    {'C00110'}
    {'C00132'}
    {'C00154'}
    {'C00159'}
    {'C00168'}
    {'C00190'}
    {'C00192'}
    {'C00201'}
    {'C00222'}
    {'C00243'}
    {'C00248'}
    {'C00301'}
    {'C00302'}
    {'C00307'}
    {'C00315'}
    {'C00322'}
    {'C00345'}
    {'C00350'}
    {'C00353'}
    {'C00389'}
    {'C00406'}
    {'C00409'}
    {'C00411'}
    {'C00446'}
    {'C00447'}
    {'C00450'}
    {'C00454'}
    {'C00458'}
    {'C00473'}
    {'C00506'}
    {'C00509'}
    {'C00522'}
    {'C00527'}
    {'C00531'}
    {'C00546'}
    {'C00561'}
    {'C00575'}
    {'C00577'}
    {'C00627'}
    {'C00647'}
    {'C00666'}
    {'C00673'}
    {'C00697'}
    {'C00700'}
    {'C00721'}
    {'C00794'}
    {'C00886'}
    {'C00887'}
    {'C00942'}
    {'C00957'}
    {'C00962'}
    {'C00963'}
    {'C00966'}
    {'C00988'}
    {'C01019'}
    {'C01041'}
    {'C01045'}
    {'C01051'}
    {'C01074'}
    {'C01094'}



In order for this to be a functional GEM, exchange reactions need to be added.


In [157]:
% Setting up the draft GEM base
%PbiviaBiomassDraftModel = importModel('Pbivia_draftGEM_v3.xml');
% create a copy to avoid overwriting
PbiviaExchangeModel = PbiviaBiomassDraftModel;
PbiviaExchangeModel.id = 'PbiviaDraft';


Perform some cleaning:


In [158]:
% deal with grRules issues
PbiviaExchangeModel = ensureGrRules(PbiviaExchangeModel);
modelKEGGgr = ensureGrRules(modelKEGG);

Patched grRules to match 11588 rxns

In [159]:
% further sanitization
modelKEGGgr = padMissingFields(modelKEGGgr, PbiviaExchangeModel);

Added geneMiriams to KEGG
Added rxnFrom to KEGG
Added metFrom to KEGG
Added geneFrom to KEGG
Added rules to KEGG
Added equations to KEGG
Added rxnReferences to KEGG
Added rxnConfidenceScores to KEGG
Added metCharges to KEGG
Added geneNames to KEGG
Added geneShortNames to KEGG



Dealing with exchange reactions:


In [160]:
% getting a cell array of metabolites to create exchange reactions for
biomassMets = getBiomassMets(PbiviaExchangeModel, 'BIOMASS_ps'); % get metabolites involved in the biomass reaction

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 306
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00009'}
    {'C00010'}
    {'C00011'}
    {'C00013'}
    {'C00014'}
    {'C00016'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00028'}
    {'C00029'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00036'}
    {'C00037'}
    {'C00041'}
    {'C00042'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00062'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00083'}
    {'C00085'}
    {'C00091'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00105'}
    {'C00109'}
    {'C00116'}
    {'C00117'}
    {'C00118'}
    {'C00120'}
    {'C00123'}
    {'C00130'}
    {'C00131'}
    {'C00

In [161]:
wasteMets = getCommonWasteMets(PbiviaExchangeModel); % get list (cell array) of common waste metabolites
essentialMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
communityMets = getCommunityExchangeMets(); % metabolites that are likely to be needed in MCMM community exchanges
allExMets = unique([biomassMets(:); wasteMets(:); essentialMets(:); communityMets(:)], 'stable'); % concatenate while avoiding potential duplicates

% add the exchange reactions to the model
PbiviaExchangeModel = addExchangeRxnsForModel(PbiviaExchangeModel, allExMets);

In [162]:

% ensure reversibility of reactions
PbiviaExchangeModel = ensureRevField(PbiviaExchangeModel);

Model already has rev field

In [163]:
sol3 = solveLP(PbiviaExchangeModel);
disp(sol3.f);

  25.210084033613445



There is now biomass production!


In [164]:
% pre-export cleaning
PbiviaExchangeModel = fixSubsystemsField(PbiviaExchangeModel);
PbiviaExchangeModel = sanitizeRxnMiriams(PbiviaExchangeModel);

Sanitized rxnMiriams for 1247 reactions



Now the model is ready for export:


In [165]:
% export the current model version
exportModel(PbiviaExchangeModel, 'Pbivia_draftGEM_v4.xml');

    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [166]:
exportToExcelFormat(PbiviaExchangeModel, 'Pbivia_draftGEM_v4.xlsx');
save('Pbivia_draftGEM_v4.mat', 'PbiviaExchangeModel');


This will be the version used for the MCMM.

## Additional tests

A few more sanity checks and tests prior to moving on to gap\-filling.


In [167]:
% Setting up the draft GEM base
%PbiviaBiomassDraftModel = importModel('Pbivia_draftGEM_v4.xml');
% create a copy to avoid overwriting
PbiviaBiomassObjDraftModel = PbiviaExchangeModel;
PbiviaBiomassObjDraftModel.id = 'PbiviaDraft';


First, checking for dead\-end metabolites:


In [168]:
deadMets = quickDeadEnds(PbiviaBiomassObjDraftModel);
fprintf('Dead-end metabolites: %d\n', numel(deadMets));

Dead-end metabolites: 0



Double\-check that the biomass metabolites are indeed present in the model:


In [169]:
biomassMets = getBiomassMets(PbiviaBiomassObjDraftModel, 'BIOMASS_ps');

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 306
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00009'}
    {'C00010'}
    {'C00011'}
    {'C00013'}
    {'C00014'}
    {'C00016'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00028'}
    {'C00029'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00036'}
    {'C00037'}
    {'C00041'}
    {'C00042'}
    {'C00044'}
    {'C00047'}
    {'C00049'}
    {'C00055'}
    {'C00058'}
    {'C00061'}
    {'C00062'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00082'}
    {'C00083'}
    {'C00085'}
    {'C00091'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00100'}
    {'C00101'}
    {'C00105'}
    {'C00109'}
    {'C00116'}
    {'C00117'}
    {'C00118'}
    {'C00120'}
    {'C00123'}
    {'C00130'}
    {'C00131'}
    {'C00

In [170]:
missing = setdiff(biomassMets, PbiviaBiomassObjDraftModel.mets);
disp(missing);


Perform a preliminary test for whether gap\-filling is necessary:


In [171]:
% block all exchange
PbiviaBiomassObjDraftModel.lb(findExcRxns(PbiviaBiomassObjDraftModel)) = 0;

% reopen minimal medium
allowMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
for i = 1:numel(allowMets)
    exIdx = find(strcmp(PbiviaBiomassObjDraftModel.mets, allowMets{i}));
    if ~isempty(exIdx)
        rxnIdx = find(PbiviaBiomassObjDraftModel.S(exIdx,:));
        PbiviaBiomassObjDraftModel.lb(rxnIdx) = -1000; % uptake allowed
    end
end

sol4 = solveLP(PbiviaBiomassObjDraftModel);
disp(sol4.f);

  25.210084033613384



The solution here is the same as before, so this looks good to go.


On Ed's recommendation, I will force the biomass flux to be positive:


In [172]:
% force the biomass flux to be positive
% Set lower and upper bounds of the biomass reaction
rxnID = 'BIOMASS_ps';
biomassIndex = find(ismember(PbiviaBiomassObjDraftModel.rxns, rxnID));

PbiviaBiomassObjDraftModel.lb(biomassIndex) = 0.1;  % minimum growth rate
PbiviaBiomassObjDraftModel.ub(biomassIndex) = 1000; % allow up to 1000


Finally, run FBA on the new model:


In [173]:
sol3 = solveLP(PbiviaBiomassObjDraftModel);
disp(sol3.f)

  25.210084033613445



Export the model:


In [174]:
exportModel(PbiviaBiomassObjDraftModel, 'Pbivia_draftGEM_v5.xml');

    R00765
    R05627_EXP_1
    R08221
    R11037
    R12896
    R12958
    R08020
    1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-19(14,15)17-18(11,12)13/h5,7H,4,6,8H2,1-3H3,(H,14,15)(H2,11,12,13)/b10-7+
Document written

In [175]:
exportToExcelFormat(PbiviaBiomassObjDraftModel, 'Pbivia_draftGEM_v5.xlsx');
save('Pbivia_draftGEM_v5.mat', 'PbiviaBiomassObjDraftModel');


## Gap\-filling attempt

On the advice of Ed, I will be doing gap\-filling from the whole of the KEGG database. While the model does technically produce biomass and so therefore is viable, it is better to add more exchanges to ensure as high\-quality of an MCMM as possible.


In [176]:
% trying gap filling
[Pbivia_filledModel, addedRxns] = fillGaps( ...
    PbiviaBiomassObjDraftModel, ...
    modelKEGGgr, ...
    false, true, true);

% Check the output of fillGaps
if isempty(Pbivia_filledModel)
    disp('fillGaps returned an empty model. Check your input models and required fields.');
elseif iscell(Pbivia_filledModel)
    disp(['fillGaps returned a cell array of size: ', mat2str(size(Pbivia_filledModel))]);
else
    disp('fillGaps returned a valid model struct.');
end

fillGaps returned an empty model. Check your input models and required fields.



Check the results:


In [177]:
% Did fillGaps return something valid?
class(Pbivia_filledModel)

ans = 'cell'

In [178]:
whos Pbivia_filledModel

  Name                    Size            Bytes  Class    Attributes
  Pbivia_filledModel      0x0                 0  cell



As before, it appears that gap\-filling is unnecessary in this case.

## Draft *Trichomonas vaginalis* GEM

Initial GEM creation:


In [179]:
%clear
%clc

% draft with KEGG
%The organisms with KEGG annotation and their associated ids
%(e.g. 'tva' for Trichomonas vaginalis) can be found from here:
%https://www.kegg.jp/kegg/catalog/org_list.html.
TvaginalisKEGGAnnotation=getKEGGModelForOrganism('tva','','','',1,0,0);

*** The model reconstruction from KEGG based on the annotation available for KEGG Species tva ***
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Pruning the model from non-tva genes... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Constructing GPR associations and annotations for the model... COMPLETE
*** Model reconstruction complete ***

In [180]:
TvaginalisKEGGHomology=getKEGGModelForOrganism('TvaginalisKEGGHMMs','trichomonas_vaginalis_GCF_026262505-1_prots_edit.fasta','euk90_kegg105','',1,0,0);

*** The model reconstruction from KEGG based on the protein homology search against KEGG Orthology specific HMMs ***
NOTE: Found euk90_kegg105 directory with pre-trained HMMs, it will therefore be used during reconstruction
Importing the global KEGG model from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggModel.mat... COMPLETE
Removing non-spontaneous reactions without GPR rules... COMPLETE
Fixing gene names in the model... COMPLETE
Importing the KEGG phylogenetic distance matrix from C:/Users/viragv/AppData/Roaming/MathWorks/MATLAB Add-Ons/Collections/RAVEN Toolbox/external/kegg/keggPhylDist.mat... COMPLETE
Generating the KEGG Orthology specific multi-FASTA files... COMPLETE
Performing clustering and multiple alignment for KEGG Orthology specific protein sets... COMPLETE
Training the KEGG Orthology specific HMMs... COMPLETE
Querying the user-specified FASTA file against the KEGG Orthology specific HMMs... COMPLETE
Parsing the HMM 

In [181]:

% merge models
TvaginalisKEGGDraftModel=mergeModels({TvaginalisKEGGAnnotation TvaginalisKEGGHomology});

% check reaction numbers
disp(numel(TvaginalisKEGGHomology.rxns)+numel(TvaginalisKEGGAnnotation.rxns));

        1490

In [182]:
disp(numel(TvaginalisKEGGDraftModel.rxns));

        1490

In [183]:

% further model cleaning
TvaginalisKEGGDraftModel=expandModel(TvaginalisKEGGDraftModel);
TvaginalisKEGGDraftModel=contractModel(TvaginalisKEGGDraftModel);


At this point, a draft GEM has been created for *T. vaginalis*, merging the information from MetaCyc and KEGG. While it needs to be improved, I'll save it for now in Excel format to avoid having to recreate it each time.


In [184]:
exportToExcelFormat(TvaginalisKEGGDraftModel, 'Tvaginalis_draftGEM_v1.xlsx');
save('Tvaginalis_draftGEM_v1.mat', 'TvaginalisKEGGDraftModel');


## Cleaning *T. vaginalis* draft GEM

In [185]:
%TvaginalisCombinedDraftModel = load('Tvaginalis_draftGEM_v1.mat');
TvaginalisCleanDraftModel = cleanGrRulesGenes(TvaginalisKEGGDraftModel);
TvaginalisCleanDraftModel = mergeDuplicateMetNames(TvaginalisCleanDraftModel);
TvaginalisCleanDraftModel = mergeDuplicateMetNames2(TvaginalisCleanDraftModel);
TvaginalisCleanDraftModel = fixSubsystemsField(TvaginalisCleanDraftModel);


The preliminary model should now be ready for export.


In [186]:
exportModel(TvaginalisCleanDraftModel, 'Tvaginalis_draftGEM_v2.xml');

    C01977
    C01978
    C02031
    C02100
    C02339
    C04157
    C04802
    C17324
    C20458
    ...and 11 more
    1S/C24H40N7O17P3S/c1-4-15(33)52-8-7-26-14(32)5-6-27-22(36)19(35)24(2,3)10-45-51(42,43)48-50(40,41)44-9-13-18(47-49(37,38)39)17(34)23(46-13)31-12-30-16-20(25)28-11-29-21(16)31/h11-13,17-19,23,34-35H,4-10H2,1-3H3,(H,26,32)(H,27,36)(H,40,41)(H,42,43)(H2,25,28,29)(H2,37,38,39)/t13-,17-,18-,19+,23-/m1/s1
Document written

In [187]:
exportToExcelFormat(TvaginalisCleanDraftModel, 'Tvaginalis_draftGEM_v2.xlsx');
save('Tvaginalis_draftGEM_v2.mat', 'TvaginalisCleanDraftModel');


Saved in SBML standard GEM format.


In [188]:
% Load the model from SBML (.xml)
%TvaginalisCleanDraftModel = importModel('Tvaginalis_draftGEM_v2.xml');


Now start quality checks:


In [189]:
%Check out if the newly generated TvaginalisCombinedDraftModel could grow
sol=solveLP(TvaginalisCleanDraftModel);
disp(sol.f);

     0

In [190]:
%The results indicate that this new model is non-functional
% ie., biomass = 0, which means there is no growth

In [191]:
% Look for reactions that produce a "biomass" metabolite
bioMets = find(contains(lower(TvaginalisCleanDraftModel.metNames), 'biomass'));

if isempty(bioMets)
    warning('No metabolites with "biomass" in the name were found.');
else
    % Find reactions involving these metabolites
    [rxnIdx, ~] = find(TvaginalisCleanDraftModel.S(bioMets,:) ~= 0);
    rxnIdx = unique(rxnIdx);
    disp('Potential biomass reactions:');
    disp([rxnIdx, TvaginalisCleanDraftModel.rxns(rxnIdx)', TvaginalisCleanDraftModel.rxnNames(rxnIdx)']);
end


## Adding a biomass reaction

Initial explorations showed that the current preliminary GEM draft doesn't have a biomass reaction. Hence, biomass will always be 0 until this is added. The following code was generated by ChatGPT; it seeds a biomass reaction from common building blocks like amino acids and nucleotides.


In [192]:
[TvaginalisBiomassDraftModel, biomassRxnID] = addPseudoBiomass_trichomonas( ...
    TvaginalisCleanDraftModel, ...
    'BIOMASS_ps', ...
    [], ...
    true, ...
    'Tvaginalis_biomass_debug.txt', ...
    'Tvaginalis_biomass_metabolites.csv' ...
);

Precursor "L-arginine" → 8 matches found
    Match: "S-adenosyl-L-methionine:[protein]-L-arginine N-methyltransferase" (ID: R11216)
        Metabolite: "S-Adenosyl-L-methionine" (ID: C00019)
        Metabolite: "Protein-L-arginine" (ID: C00613)
    Match: "S-adenosyl-L-methionine:[protein]-N(omega)-methyl-L-arginine N-methyltransferase ([protein]-N(omega),N(omega)-dimethyl-L-arginine-forming)" (ID: R11217)
        Metabolite: "S-Adenosyl-L-methionine" (ID: C00019)
        Metabolite: "Protein N(omega)-methyl-L-arginine" (ID: C04143)
    Match: "S-adenosyl-L-methionine:[protein]-N(omega)-methyl-L-arginine N-methyltransferase ([protein]-N(omega),N(omega)'-dimethyl-L-arginine-forming)" (ID: R11218)
        Metabolite: "S-Adenosyl-L-methionine" (ID: C00019)
        Metabolite: "Protein N(omega)-methyl-L-arginine" (ID: C04143)
    Match: "S-adenosyl-L-methionine:[protein]-L-arginine N-methyltransferase ([protein]-N(omega),N(omega)-dimethyl-L-arginine-forming)" (ID: R11219)
        Metabolit



Quick cleaning to ensure RAVEN structures are correctly kept:


In [193]:
TvaginalisBiomassDraftModel = syncModelStruct(TvaginalisBiomassDraftModel, 'Tvaginalis_biomass_sync_log.csv');

syncModelStruct: Log saved to "Tvaginalis_biomass_sync_log.csv"

In [194]:
checkModelCoreConsistency(TvaginalisBiomassDraftModel);

--- BASIC DIMENSION CHECKS ---
Reactions: 891
Metabolites: 1160
Genes: 1734
Empty grRules in 145 reactions

In [195]:
TvaginalisBiomassDraftModel = sanitizeGEMforSBML(TvaginalisBiomassDraftModel);
TvaginalisBiomassDraftModel = fixRxnConfidenceScores(TvaginalisBiomassDraftModel);


I will now save the new model with the biomass reaction included:


In [196]:
exportModel(TvaginalisBiomassDraftModel, 'Tvaginalis_draftGEM_v3.xml');

    1S/C24H40N7O17P3S/c1-4-15(33)52-8-7-26-14(32)5-6-27-22(36)19(35)24(2,3)10-45-51(42,43)48-50(40,41)44-9-13-18(47-49(37,38)39)17(34)23(46-13)31-12-30-16-20(25)28-11-29-21(16)31/h11-13,17-19,23,34-35H,4-10H2,1-3H3,(H,26,32)(H,27,36)(H,40,41)(H,42,43)(H2,25,28,29)(H2,37,38,39)/t13-,17-,18-,19+,23-/m1/s1
Document written

In [197]:
exportToExcelFormat(TvaginalisBiomassDraftModel, 'Tvaginalis_draftGEM_v3.xlsx');
save('Tvaginalis_draftGEM_v3.mat', 'TvaginalisBiomassDraftModel');


Now that a biomass reaction has been added to the model, I will try to solve again:


In [198]:
%TvaginalisBiomassDraftModel = importModel('Tvaginalis_draftGEM_v3.xml');
sol2 = solveLP(TvaginalisBiomassDraftModel);
disp(sol2.f)

     0



Still 0, so still no growth.

## Addition of exchange reactions

The simplified gap report:


In [199]:
% taking a look at the gap situation
%gapReport(TvaginalisBiomassDraftModel);
gapReport_vSimple(TvaginalisBiomassDraftModel);

Dead-end metabolites: 572
    {'C00012'}
    {'C00016'}
    {'C00018'}
    {'C00053'}
    {'C00054'}
    {'C00058'}
    {'C00071'}
    {'C00114'}
    {'C00134'}
    {'C00169'}
    {'C00178'}
    {'C00190'}
    {'C00191'}
    {'C00194'}
    {'C00203'}
    {'C00227'}
    {'C00229'}
    {'C00237'}
    {'C00243'}
    {'C00248'}
    {'C00263'}
    {'C00302'}
    {'C00322'}
    {'C00344'}
    {'C00389'}
    {'C00409'}
    {'C00412'}
    {'C00415'}
    {'C00422'}
    {'C00427'}
    {'C00447'}
    {'C00450'}
    {'C00458'}
    {'C00509'}
    {'C00527'}
    {'C00561'}
    {'C00584'}
    {'C00627'}
    {'C00647'}
    {'C00683'}
    {'C00693'}
    {'C00700'}
    {'C00721'}
    {'C00804'}
    {'C00818'}
    {'C00857'}
    {'C00886'}
    {'C00940'}
    {'C00962'}
    {'C00963'}
    {'C00988'}
    {'C01019'}
    {'C01024'}
    {'C01041'}
    {'C01094'}
    {'C01133'}
    {'C01168'}
    {'C01181'}
    {'C01233'}
    {'C01234'}
    {'C01246'}
    {'C01252'}
    {'C01262'}
    {'C01290'}
    {'C01297'}



In order for this to be a functional GEM, exchange reactions need to be added.


In [200]:
% Setting up the draft GEM base
%TvaginalisBiomassDraftModel = importModel('Tvaginalis_draftGEM_v3.xml');
% create a copy to avoid overwriting
TvaginalisExchangeModel = TvaginalisBiomassDraftModel;
TvaginalisExchangeModel.id = 'TvaginalisDraft';


Perform some cleaning:


In [201]:
% deal with grRules issues
TvaginalisExchangeModel = ensureGrRules(TvaginalisExchangeModel);
modelKEGGgr = ensureGrRules(modelKEGG);

Patched grRules to match 11588 rxns

In [202]:
% further sanitization
modelKEGGgr = padMissingFields(modelKEGGgr, TvaginalisExchangeModel);

Added geneMiriams to KEGG
Added rxnFrom to KEGG
Added metFrom to KEGG
Added geneFrom to KEGG
Added rules to KEGG
Added equations to KEGG
Added rxnReferences to KEGG
Added rxnConfidenceScores to KEGG
Added metCharges to KEGG
Added geneNames to KEGG
Added geneShortNames to KEGG



Dealing with exchange reactions:


In [203]:
% getting a cell array of metabolites to create exchange reactions for
biomassMets = getBiomassMets(TvaginalisExchangeModel, 'BIOMASS_ps'); % get metabolites involved in the biomass reaction

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 358
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00011'}
    {'C00013'}
    {'C00014'}
    {'C00015'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00029'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00035'}
    {'C00036'}
    {'C00037'}
    {'C00040'}
    {'C00042'}
    {'C00044'}
    {'C00049'}
    {'C00055'}
    {'C00061'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00081'}
    {'C00082'}
    {'C00084'}
    {'C00085'}
    {'C00092'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00101'}
    {'C00103'}
    {'C00104'}
    {'C00105'}
    {'C00110'}
    {'C00112'}
    {'C00116'}
    {'C00117'}
    {'C00118'}
    {'C00121'}
    {'C00123'}
    {'C00124'}
    {'C00

In [204]:
wasteMets = getCommonWasteMets(TvaginalisExchangeModel); % get list (cell array) of common waste metabolites
essentialMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
communityMets = getCommunityExchangeMets(); % metabolites that are likely to be needed in MCMM community exchanges
allExMets = unique([biomassMets(:); wasteMets(:); essentialMets(:); communityMets(:)], 'stable'); % concatenate while avoiding potential duplicates

% add the exchange reactions to the model
TvaginalisExchangeModel = addExchangeRxnsForModel(TvaginalisExchangeModel, allExMets);

In [205]:

% ensure reversibility of reactions
TvaginalisExchangeModel = ensureRevField(TvaginalisExchangeModel);

Model already has rev field

In [206]:
sol3 = solveLP(TvaginalisExchangeModel);
disp(sol3.f);

  19.108280254777071



There is now biomass production!


In [207]:
% pre-export cleaning
TvaginalisExchangeModel = fixSubsystemsField(TvaginalisExchangeModel);
TvaginalisExchangeModel = sanitizeRxnMiriams(TvaginalisExchangeModel);

Sanitized rxnMiriams for 1251 reactions



Now the model is ready for export:


In [208]:
% export the current model version
exportModel(TvaginalisExchangeModel, 'Tvaginalis_draftGEM_v4.xml');

    1S/C24H40N7O17P3S/c1-4-15(33)52-8-7-26-14(32)5-6-27-22(36)19(35)24(2,3)10-45-51(42,43)48-50(40,41)44-9-13-18(47-49(37,38)39)17(34)23(46-13)31-12-30-16-20(25)28-11-29-21(16)31/h11-13,17-19,23,34-35H,4-10H2,1-3H3,(H,26,32)(H,27,36)(H,40,41)(H,42,43)(H2,25,28,29)(H2,37,38,39)/t13-,17-,18-,19+,23-/m1/s1
Document written

In [209]:
exportToExcelFormat(TvaginalisExchangeModel, 'Tvaginalis_draftGEM_v4.xlsx');
save('Tvaginalis_draftGEM_v4.mat', 'TvaginalisExchangeModel');


This will be the version used for the MCMM.

## Additional tests

A few more sanity checks and tests prior to moving on to gap\-filling.


In [210]:
% Setting up the draft GEM base
%TvaginalisBiomassDraftModel = importModel('Tvaginalis_draftGEM_v4.xml');
% create a copy to avoid overwriting
TvaginalisBiomassObjDraftModel = TvaginalisExchangeModel;
TvaginalisBiomassObjDraftModel.id = 'TvaginalisDraft';


First, checking for dead\-end metabolites:


In [211]:
deadMets = quickDeadEnds(TvaginalisBiomassObjDraftModel);
fprintf('Dead-end metabolites: %d\n', numel(deadMets));

Dead-end metabolites: 0



Double\-check that the biomass metabolites are indeed present in the model:


In [212]:
biomassMets = getBiomassMets(TvaginalisBiomassObjDraftModel, 'BIOMASS_ps');

Biomass reaction: BIOMASS_ps
Number of metabolites in biomass reaction: 358
    {'C00001'}
    {'C00002'}
    {'C00003'}
    {'C00004'}
    {'C00005'}
    {'C00006'}
    {'C00007'}
    {'C00008'}
    {'C00009'}
    {'C00010'}
    {'C00011'}
    {'C00013'}
    {'C00014'}
    {'C00015'}
    {'C00019'}
    {'C00020'}
    {'C00022'}
    {'C00024'}
    {'C00025'}
    {'C00026'}
    {'C00029'}
    {'C00030'}
    {'C00031'}
    {'C00033'}
    {'C00035'}
    {'C00036'}
    {'C00037'}
    {'C00040'}
    {'C00042'}
    {'C00044'}
    {'C00049'}
    {'C00055'}
    {'C00061'}
    {'C00063'}
    {'C00064'}
    {'C00065'}
    {'C00073'}
    {'C00075'}
    {'C00078'}
    {'C00079'}
    {'C00080'}
    {'C00081'}
    {'C00082'}
    {'C00084'}
    {'C00085'}
    {'C00092'}
    {'C00093'}
    {'C00095'}
    {'C00097'}
    {'C00101'}
    {'C00103'}
    {'C00104'}
    {'C00105'}
    {'C00110'}
    {'C00112'}
    {'C00116'}
    {'C00117'}
    {'C00118'}
    {'C00121'}
    {'C00123'}
    {'C00124'}
    {'C00

In [213]:
missing = setdiff(biomassMets, TvaginalisBiomassObjDraftModel.mets);
disp(missing);


Perform a preliminary test for whether gap\-filling is necessary:


In [214]:
% block all exchange
TvaginalisBiomassObjDraftModel.lb(findExcRxns(TvaginalisBiomassObjDraftModel)) = 0;

% reopen minimal medium
allowMets = {'C00031','C00014','C00007','C00009','C00036'}; % glucose, ammonium, oxygen, phosphate, sulfate
for i = 1:numel(allowMets)
    exIdx = find(strcmp(TvaginalisBiomassObjDraftModel.mets, allowMets{i}));
    if ~isempty(exIdx)
        rxnIdx = find(TvaginalisBiomassObjDraftModel.S(exIdx,:));
        TvaginalisBiomassObjDraftModel.lb(rxnIdx) = -1000; % uptake allowed
    end
end

sol4 = solveLP(TvaginalisBiomassObjDraftModel);
disp(sol4.f);

  19.108280254777128



The solution here is the same as before, so this looks good to go.


On Ed's recommendation, I will force the biomass flux to be positive:


In [215]:
% force the biomass flux to be positive
% Set lower and upper bounds of the biomass reaction
rxnID = 'BIOMASS_ps';
biomassIndex = find(ismember(TvaginalisBiomassObjDraftModel.rxns, rxnID));

TvaginalisBiomassObjDraftModel.lb(biomassIndex) = 0.1;  % minimum growth rate
TvaginalisBiomassObjDraftModel.ub(biomassIndex) = 1000; % allow up to 1000


Finally, run FBA on the new model:


In [216]:
sol3 = solveLP(TvaginalisBiomassObjDraftModel);
disp(sol3.f)

  19.108280254777071



Export the model:


In [217]:
exportModel(TvaginalisBiomassObjDraftModel, 'Tvaginalis_draftGEM_v5.xml');

    R00078
    R00765
    R07942
    R07978
    R07979
    R08221_EXP_1
    R08235
    R12958
    R12989
    ...and 6 more
    1S/C24H40N7O17P3S/c1-4-15(33)52-8-7-26-14(32)5-6-27-22(36)19(35)24(2,3)10-45-51(42,43)48-50(40,41)44-9-13-18(47-49(37,38)39)17(34)23(46-13)31-12-30-16-20(25)28-11-29-21(16)31/h11-13,17-19,23,34-35H,4-10H2,1-3H3,(H,26,32)(H,27,36)(H,40,41)(H,42,43)(H2,25,28,29)(H2,37,38,39)/t13-,17-,18-,19+,23-/m1/s1
Document written

In [218]:
exportToExcelFormat(TvaginalisBiomassObjDraftModel, 'Tvaginalis_draftGEM_v5.xlsx');
save('Tvaginalis_draftGEM_v5.mat', 'TvaginalisBiomassObjDraftModel');


## Gap\-filling attempt

On the advice of Ed, I will be doing gap\-filling from the whole of the KEGG database. While the model does technically produce biomass and so therefore is viable, it is better to add more exchanges to ensure as high\-quality of an MCMM as possible.


In [219]:
% trying gap filling
[Tvaginalis_filledModel, addedRxns] = fillGaps( ...
    TvaginalisBiomassObjDraftModel, ...
    modelKEGGgr, ...
    false, true, true);

% Check the output of fillGaps
if isempty(Tvaginalis_filledModel)
    disp('fillGaps returned an empty model. Check your input models and required fields.');
elseif iscell(Tvaginalis_filledModel)
    disp(['fillGaps returned a cell array of size: ', mat2str(size(Tvaginalis_filledModel))]);
else
    disp('fillGaps returned a valid model struct.');
end

fillGaps returned an empty model. Check your input models and required fields.



Check the results:


In [220]:
% Did fillGaps return something valid?
class(Tvaginalis_filledModel)

ans = 'cell'

In [221]:
whos Tvaginalis_filledModel

  Name                        Size            Bytes  Class    Attributes
  Tvaginalis_filledModel      0x0                 0  cell



As before, it appears that gap\-filling is unnecessary in this case.

## Human GEM

The human GEM that I will be using was downloaded from GitHub: [https://github.com/SysBioChalmers/Human\-GEM](https://github.com/SysBioChalmers/Human-GEM)


The SBML\-formatted version is found at: `Human-GEM/model/Human-GEM.xml`


Unfortunately, Human\-GEM uses BIGG reaction and metabolite names, with clashes with the KEGG\-derived draft GEMs created throughout the rest of this workflow. Therefore, prior to constructing the MCMMs, I will create a version of Human\-GEM that uses KEGG reaction and metabolite names.


I began by downloading the ChEBI FTP flat files and parsing it to extract ChEBI\-KEGG pairs:


 `wget` [`ftp://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/database_accession.tsv`](ftp://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/database_accession.tsv)


`awk -F'\\t' '\$3=="KEGG COMPOUND" && \$4=="KEGG COMPOUND accession" {print "CHEBI:" \$2 "\\t" \$5}' database_accession.tsv > chebi_to_kegg.tsv`


Now using this file to create a KEGG version of Human\-GEM:


In [222]:
%% --- Load Human-GEM XML ---
xmlFile = 'Human-GEM.xml';
xDoc = xmlread(xmlFile);

%% --- Load ChEBI -> KEGG mapping ---
chebiMap = readtable('chebi_to_kegg.tsv','FileType','text','Delimiter','\t',...
    'ReadVariableNames',false);
chebiMap.Properties.VariableNames = {'ChEBI','KEGG'};

%% --- Extract all metabolites ---
metabolites = xDoc.getElementsByTagName('species');
numMets = metabolites.getLength;

% Storage for IDs
metID   = cell(numMets,1);
metName = cell(numMets,1);
keggID  = cell(numMets,1);
chebiID = cell(numMets,1);

for i = 0:numMets-1
    node = metabolites.item(i);
    metID{i+1}   = char(node.getAttribute('id'));
    metName{i+1} = char(node.getAttribute('name'));

    % Extract KEGG and ChEBI URIs
    annotationNode = node.getElementsByTagName('annotation').item(0);
    if ~isempty(annotationNode)
        rdfNode = annotationNode.getElementsByTagName('rdf:RDF').item(0);
        if ~isempty(rdfNode)
            liNodes = rdfNode.getElementsByTagName('rdf:li');
            for j = 0:liNodes.getLength-1
                uri = char(liNodes.item(j).getAttribute('rdf:resource'));
                if contains(uri,'kegg.compound/')
                    keggID{i+1} = extractAfter(uri,'kegg.compound/');
                elseif contains(uri,'chebi/')
                    chebiID{i+1} = extractAfter(uri,'chebi/');
                end
            end
        end
    end
end

%% --- Fill missing KEGG IDs via ChEBI mapping ---
for i = 1:numMets
    if isempty(keggID{i}) && ~isempty(chebiID{i})
        idx = find(strcmp(chebiMap.ChEBI, chebiID{i}));
        if ~isempty(idx)
            keggID{i} = chebiMap.KEGG{idx(1)};
        end
    end
end

%% --- Normalize KEGG IDs to strings ---
% Replace [] with '' and enforce cellstr
for i = 1:numel(keggID)
    if isempty(keggID{i})
        keggID{i} = '';
    else
        keggID{i} = char(keggID{i});
    end
end

%% --- Handle duplicate KEGG IDs: keep first, log others ---
[uniqueKegg, ia, ic] = unique(keggID, 'stable');
dupIdx = setdiff(1:numel(keggID), ia);

if ~isempty(dupIdx)
    dupLog = 'duplicate_metabolites.log';
    fid = fopen(dupLog,'w');
    fprintf(fid,'The following KEGG IDs had multiple matches. First kept, others removed:\n\n');
    dupGroups = accumarray(ic(dupIdx), dupIdx, [], @(x){x});
    for g = 1:numel(dupGroups)
        if isempty(dupGroups{g})
            continue   % skip this group
        end

        idxs = dupGroups{g};   % all indices in this group
        refKegg = keggID{idxs(1)};   % use first as reference

        % Example: check consistency
        if ~all(strcmp(refKegg, keggID(idxs)))
            fprintf('Inconsistent KEGG IDs in group %d\n', g);
        end
    end
    fclose(fid);
    fprintf('Warning: %d duplicate KEGG IDs found. See %s for details.\n', ...
        numel(dupIdx), dupLog);
end

In [223]:

% Keep only first occurrence of each KEGG ID
keggID  = uniqueKegg;
metID   = metID(ia);
metName = metName(ia);
chebiID = chebiID(ia);
numMets = numel(metID);

%% --- Build mapping oldID -> KEGG ID (no compartments) ---
validIdx = ~cellfun(@isempty, keggID);
old2kegg = containers.Map(metID(validIdx), keggID(validIdx));

%% --- Update XML species nodes with plain KEGG IDs ---
for i = 0:metabolites.getLength-1
    node = metabolites.item(i);
    oldID = char(node.getAttribute('id'));

    if isKey(old2kegg, oldID) && ~isempty(old2kegg(oldID))
        newKEGG = old2kegg(oldID);

        % Use plain KEGG ID
        node.setAttribute('id', newKEGG);
        node.setAttribute('name', newKEGG);

        % Update RDF annotation with KEGG URI
        annotationNode = node.getElementsByTagName('annotation').item(0);
        if ~isempty(annotationNode)
            rdfNode = annotationNode.getElementsByTagName('rdf:RDF').item(0);
            if ~isempty(rdfNode)
                bagNode = rdfNode.getElementsByTagName('rdf:Bag').item(0);
                liNodes = bagNode.getElementsByTagName('rdf:li');

                % Remove old KEGG URIs
                for j = liNodes.getLength-1:-1:0
                    uri = char(liNodes.item(j).getAttribute('rdf:resource'));
                    if contains(uri,'kegg.compound/')
                        bagNode.removeChild(liNodes.item(j));
                    end
                end

                % Add new KEGG URI
                docNode = xDoc.createElement('rdf:li');
                docNode.setAttribute('rdf:resource', ['https://identifiers.org/kegg.compound/' newKEGG]);
                bagNode.appendChild(docNode);
            end
        end
    end
end

%% --- Update reactions using new KEGG IDs ---
missingMets = {};
reactions = xDoc.getElementsByTagName('reaction');
for i = 0:reactions.getLength-1
    rNode = reactions.item(i);
    speciesRefs = rNode.getElementsByTagName('speciesReference');
    for j = 0:speciesRefs.getLength-1
        sRef = speciesRefs.item(j);
        oldMet = char(sRef.getAttribute('species'));
        if isKey(old2kegg, oldMet) && ~isempty(old2kegg(oldMet))
            sRef.setAttribute('species', old2kegg(oldMet));
        else
            missingMets{end+1,1} = oldMet; %#ok<SAGROW>
        end
    end
end

%% --- Fix exchange reaction IDs ---
for i = 0:reactions.getLength-1
    rNode = reactions.item(i);
    speciesRefs = rNode.getElementsByTagName('speciesReference');

    if speciesRefs.getLength == 1
        sRef = speciesRefs.item(0);
        oldMet = char(sRef.getAttribute('species'));

        if isKey(old2kegg, oldMet)
            newMet = old2kegg(oldMet);

            % Update the metabolite reference
            sRef.setAttribute('species', newMet);

            % Rename the reaction itself: EX_<KEGG>
            newRxnID = ['EX_' newMet];
            rNode.setAttribute('id', newRxnID);
            rNode.setAttribute('name', ['Exchange for ' newMet]);
        end
    end
end

%% --- Write missing metabolites to log file ---
missingMets = unique(missingMets);
if ~isempty(missingMets)
    logFile = 'missing_metabolites.log';
    fid = fopen(logFile,'w');
    fprintf(fid,'The following metabolites were not found in the KEGG mapping:\n');
    fprintf(fid,'%s\n',missingMets{:});
    fclose(fid);
    fprintf('Warning: %d metabolites missing KEGG IDs. See %s for details.\n', ...
        numel(missingMets), logFile);
end

In [224]:

%% --- Save KEGG XML ---
keggXMLfile = 'Human-GEM_KEGG.xml';
xmlwrite(keggXMLfile, xDoc);


Finally, ensure compatibility with RAVEN/COBRA:


In [225]:
%% --- Import back in RAVEN to test ---
model = importModel(keggXMLfile);

The model contains 0 errors and 1 warnings.
    1/C10H12N2O3/c11-7-4-2-1-3-6(7)9(13)5-8(12)10(14)15/h1-4,8H,5,11-12H2,(H,14,15)/t8-/s2
    1/C10H12N2O4/c11-6(10(15)16)4-8(14)5-2-1-3-7(13)9(5)12/h1-3,6,13H,4,11-12H2,(H,15,16)/t6-/s2
    1/C10H17NO3/c1-11-6-3-4-7(11)9(8(12)5-6)10(13)14-2/h6-9,12H,3-5H2,1-2H3/t6-,7+,8-,9+/s2
    1/C10H19NO4/c1-5-10(14)15-8(6-9(12)13)7-11(2,3)4/h8H,5-7H2,1-4H3
    1/C11H21NO4/c1-5-6-11(15)16-9(7-10(13)14)8-12(2,3)4/h9H,5-8H2,1-4H3
    1/C14H20N6O5S/c15-6(14(23)24)1-2-26-3-7-9(21)10(22)13(25-7)20-5-19-8-11(16)17-4-18-12(8)20/h4-7,9-10,13,21-22H,1-3,15H2,(H,23,24)(H2,16,17,18)/t6-,7+,9+,10+,13+/s2
    1/C17H21NO4/c1-18-12-8-9-13(18)15(17(20)21-2)14(10-12)22-16(19)11-6-4-3-5-7-11/h3-7,12-15H,8-10H2,1-2H3/t12-,13+,14-,15+/s2
    1/C20H12O/c1-2-11-4-5-13-10-14-7-9-16-20(21-16)19(14)15-8-6-12(3-1)17(11)18(13)15/h1-10,16,20H
    1/C20H12O/c1-2-6-13-12(4-1)10-16-18-14(13)9-8-11-5-3-7-15(17(11)18)19-20(16)21-19/h1-10,19-20H
    ...and 147 more

In [226]:
exportModel(model,'Human-GEM_KEGG_test.xml');  % re-export to check formatting

    1/C10H12N2O3/c11-7-4-2-1-3-6(7)9(13)5-8(12)10(14)15/h1-4,8H,5,11-12H2,(H,14,15)/t8-/s2
    1/C10H12N2O4/c11-6(10(15)16)4-8(14)5-2-1-3-7(13)9(5)12/h1-3,6,13H,4,11-12H2,(H,15,16)/t6-/s2
    1/C10H17NO3/c1-11-6-3-4-7(11)9(8(12)5-6)10(13)14-2/h6-9,12H,3-5H2,1-2H3/t6-,7+,8-,9+/s2
    1/C10H19NO4/c1-5-10(14)15-8(6-9(12)13)7-11(2,3)4/h8H,5-7H2,1-4H3
    1/C11H21NO4/c1-5-6-11(15)16-9(7-10(13)14)8-12(2,3)4/h9H,5-8H2,1-4H3
    1/C14H20N6O5S/c15-6(14(23)24)1-2-26-3-7-9(21)10(22)13(25-7)20-5-19-8-11(16)17-4-18-12(8)20/h4-7,9-10,13,21-22H,1-3,15H2,(H,23,24)(H2,16,17,18)/t6-,7+,9+,10+,13+/s2
    1/C17H21NO4/c1-18-12-8-9-13(18)15(17(20)21-2)14(10-12)22-16(19)11-6-4-3-5-7-11/h3-7,12-15H,8-10H2,1-2H3/t12-,13+,14-,15+/s2
    1/C20H12O/c1-2-11-4-5-13-10-14-7-9-16-20(21-16)19(14)15-8-6-12(3-1)17(11)18(13)15/h1-10,16,20H
    1/C20H12O/c1-2-6-13-12(4-1)10-16-18-14(13)9-8-11-5-3-7-15(17(11)18)19-20(16)21-19/h1-10,19-20H
    ...and 147 more
Document written

In [227]:

disp('Human-GEM KEGG version created and validated with RAVEN import/export.');

Human-GEM KEGG version created and validated with RAVEN import/export.


## Conclusions

With this, all preliminary draft GEMs have been generated. The MCMM creation part of the pipeline will be handled in Jupyter. I will be using the \*\_draftGEM\_v4.xml files for the MCMM generation. I will use the KEGG version of the

